In [104]:
import gc
import os
import json
import pickle
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from datetime import date
from bson import ObjectId
import matplotlib.dates as md
from datetime import datetime
from dotenv import load_dotenv
from pymongo import MongoClient
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

Connect securely to the database

In [105]:
load_dotenv("config.env")
MONGO_USER = os.getenv("MONGO_USER")
MONGO_PASSWORD = os.getenv("MONGO_PASSWORD")
client = MongoClient("mongodb://" + MONGO_USER + ":" + MONGO_PASSWORD + "@localhost:27017/")
database = client.rais

Find all the users

In [106]:
users = database.fitbit.distinct('id')

Integrate all the fitbit data one by one with daily granularity

Nightly Temperature

In [107]:
# Reading temperature data from MongoDB
users_skin_temperature = pd.DataFrame(columns=["id", "data"])
for user in users:
    user_data = pd.DataFrame(list(
                    db.fitbit.find({ "$and": [
                        { "type": "Computed Temperature"},
                        {"data.type": "SKIN"},
                        {"id": user}
                    ] },
                        {"id": 1, "data.nightly_temperature": 1, "data.sleep_end": 1, "_id": 0}
                    )
                ))

    users_skin_temperature = pd.concat([users_skin_temperature, user_data], axis=0)

# split data column (json format) into two columns
users_skin_temperature["date"] = users_skin_temperature["data"].apply(lambda d: d["sleep_end"])
users_skin_temperature["nightly_temperature"] = users_skin_temperature["data"].apply(lambda d: d["nightly_temperature"])
users_skin_temperature.drop(["data"], inplace=True, axis=1)

# convert timestamp date object and then to datetime64[ns]
users_skin_temperature["date"] = pd.to_datetime(pd.to_datetime(users_skin_temperature["date"]).dt.date)

# merge with df
df = users_skin_temperature

# checking for duplicates; if the two values are identical it means that all duplicates (in terms of date and ID) have also equal values.
df = df.groupby(["id", "date"]).first()
df

nightly_temperature
id                       date                           
621e2e8e67b776a24055b564 2021-05-24            34.137687
                         2021-05-25            33.794544
                         2021-05-26            34.611011
                         2021-05-27            34.408304
                         2021-05-28            34.178922
...                                                  ...
621e375b67b776a240290cdc 2021-07-24            33.687826
                         2021-07-25            34.112386
                         2021-07-26            33.895137
                         2021-07-27            33.758319
                         2021-07-29            34.571786

[3359 rows x 1 columns]

Heart Rate Variability

In [108]:
users_HRV = pd.DataFrame(columns=["id", "data"])
for user in users:
    user_data = pd.DataFrame(list(
        db.fitbit.find({"$and": [
            {"type": "Daily Heart Rate Variability Summary"},
            {"id": user}
        ]},
            {"id": 1, "data.timestamp": 1, "data.nremhr": 1, "data.rmssd": 1, "_id": 0}
        )
    ))

    users_HRV = pd.concat([users_HRV, user_data], axis=0)

# split data column (json format) into two columns
users_HRV["date"] = users_HRV["data"].apply(lambda d: d["timestamp"])
users_HRV["nremhr"] = users_HRV["data"].apply(lambda d: d["nremhr"])
users_HRV["rmssd"] = users_HRV["data"].apply(lambda d: d["rmssd"])
users_HRV.drop(["data"], inplace=True, axis=1)

# convert timestamp date object and then to datetime64[ns]
users_HRV["date"] = pd.to_datetime(pd.to_datetime(users_HRV["date"]).dt.date)

# merge
df = df.merge(users_HRV, how='outer', on=['id', 'date'])
df

,id,date,nightly_temperature,nremhr,rmssd
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603
1,621e2e8e67b776a24055b564,2021-05-25,33.794544,57.681,94.303
2,621e2e8e67b776a24055b564,2021-05-26,34.611011,57.481,119.212
3,621e2e8e67b776a24055b564,2021-05-27,34.408304,57.493,111.709
4,621e2e8e67b776a24055b564,2021-05-28,34.178922,56.750,103.034
...,...,...,...,...,...
3433,621e356967b776a24027bd9f,2022-01-15,NaN,63.064,55.949
3434,621e356967b776a24027bd9f,2022-01-19,NaN,59.473,56.795
3435,621e360b67b776a24039709f,2021-05-24,NaN,68.321,31.413
3436,621e367e67b776a24087d75d,2022-01-22,NaN,63.564,31.473


SpO2

In [109]:
users_spo2 = pd.DataFrame(columns=["id", "data"])
for user in users:
    user_data = pd.DataFrame(list(
        db.fitbit.find({"$and": [
            {"type": "Daily SpO2"},
            {"id": user}
        ]},
            {"id": 1, "data.timestamp": 1, "data.average_value": 1, "_id": 0}
        )
    ))

    users_spo2 = pd.concat([users_spo2, user_data], axis=0)

# split data column (json format) into two columns
users_spo2["date"] = users_spo2["data"].apply(lambda d: d["timestamp"])
users_spo2["spo2"] = users_spo2["data"].apply(lambda d: d["average_value"])
users_spo2.drop(["data"], inplace=True, axis=1)

# convert timestamp date object and then to datetime64[ns]
users_spo2["date"] = pd.to_datetime(pd.to_datetime(users_spo2["date"]).dt.date)

# merge
df = df.merge(users_spo2, how='outer', on=['id', 'date'])

# drop duplicates
df = df.groupby(["id", "date"]).first()
df

nightly_temperature  nremhr    rmssd  \
id                       date                                               
621e2e8e67b776a24055b564 2021-05-24            34.137687  57.432   89.603   
                         2021-05-25            33.794544  57.681   94.303   
                         2021-05-26            34.611011  57.481  119.212   
                         2021-05-27            34.408304  57.493  111.709   
                         2021-05-28            34.178922  56.750  103.034   
...                                                  ...     ...      ...   
621e375b67b776a240290cdc 2021-07-24            33.687826  69.579   19.407   
                         2021-07-25            34.112386  65.899   22.892   
                         2021-07-26            33.895137  65.468   22.888   
                         2021-07-27            33.758319  63.808   21.189   
                         2021-07-29            34.571786  63.397   20.818   

                                     spo2  
id                       date              
621e2e8e67b776a24055b564 2021-05-24   NaN  
                         2021-05-25   NaN  
                         2021-05-26   NaN  
                         2021-05-27   NaN  
                         2021-05-28   NaN  
...                                   ...  
621e375b67b776a240290cdc 2021-07-24   NaN  
                         2021-07-25   NaN  
                         2021-07-26   NaN  
                         2021-07-27   NaN  
                         2021-07-29   NaN  

[3475 rows x 4 columns]

Respiratory Rate Summary

In [110]:
users_respiratory_rate = pd.DataFrame(columns=["id", "data"])
for user in users:
    user_data = pd.DataFrame(list(
        db.fitbit.find({"$and": [
            {"type": "Respiratory Rate Summary"},
            {"id": user}
        ]},
            {"id": 1, "data.timestamp": 1, "data.full_sleep_breathing_rate": 1, "_id": 0}
        )
    ))

    users_respiratory_rate = pd.concat([users_respiratory_rate, user_data], axis=0)

# split data column (json format) into two columns
users_respiratory_rate["date"] = users_respiratory_rate["data"].apply(lambda d: d["timestamp"])
users_respiratory_rate["full_sleep_breathing_rate"] = users_respiratory_rate["data"].apply(lambda d: d["full_sleep_breathing_rate"])
users_respiratory_rate.drop(["data"], inplace=True, axis=1)

# convert timestamp date object and then to datetime64[ns]
users_respiratory_rate["date"] = pd.to_datetime(pd.to_datetime(users_respiratory_rate["date"]).dt.date)

# merge
df = df.merge(users_respiratory_rate, how='outer', on=['id', 'date'])

# drop duplicates
df = df.groupby(["id", "date"]).max()
df

nightly_temperature  nremhr    rmssd  \
id                       date                                               
621e2e8e67b776a24055b564 2021-05-24            34.137687  57.432   89.603   
                         2021-05-25            33.794544  57.681   94.303   
                         2021-05-26            34.611011  57.481  119.212   
                         2021-05-27            34.408304  57.493  111.709   
                         2021-05-28            34.178922  56.750  103.034   
...                                                  ...     ...      ...   
621e375b67b776a240290cdc 2021-07-24            33.687826  69.579   19.407   
                         2021-07-25            34.112386  65.899   22.892   
                         2021-07-26            33.895137  65.468   22.888   
                         2021-07-27            33.758319  63.808   21.189   
                         2021-07-29            34.571786  63.397   20.818   

                                     spo2  full_sleep_breathing_rate  
id                       date                                         
621e2e8e67b776a24055b564 2021-05-24   NaN                       14.8  
                         2021-05-25   NaN                       15.8  
                         2021-05-26   NaN                       14.6  
                         2021-05-27   NaN                       14.8  
                         2021-05-28   NaN                       15.2  
...                                   ...                        ...  
621e375b67b776a240290cdc 2021-07-24   NaN                       17.0  
                         2021-07-25   NaN                       18.0  
                         2021-07-26   NaN                       17.0  
                         2021-07-27   NaN                       17.4  
                         2021-07-29   NaN                       16.8  

[3476 rows x 5 columns]

Stress Score

In [111]:
users_stress = pd.DataFrame(columns=["id", "data"])
for user in users:
    user_data = pd.DataFrame(list(
        db.fitbit.find({"$and": [
            {"type": "Stress Score"},
            {"id": user}
        ]},
            {"id": 1, "data.DATE": 1, "data.STRESS_SCORE": 1, "data.SLEEP_POINTS": 1, "data.MAX_SLEEP_POINTS": 1, "data.EXERTION_POINTS": 1, "data.MAX_EXERTION_POINTS": 1, "data.RESPONSIVENESS_POINTS": 1, "data.MAX_RESPONSIVENESS_POINTS": 1, "_id": 0}
        )
    ))

    users_stress = pd.concat([users_stress, user_data], axis=0)

users_avg_max_sleep_points = np.mean(users_stress["data"].apply(lambda d: d["MAX_SLEEP_POINTS"]))
users_avg_max_exertion_points = np.mean(users_stress["data"].apply(lambda d: d["MAX_EXERTION_POINTS"]))
users_avg_max_responsiveness_points = np.mean(users_stress["data"].apply(lambda d: d["MAX_RESPONSIVENESS_POINTS"]))

# split data column (json format) into two columns
users_stress["date"] = users_stress["data"].apply(lambda d: d["DATE"])
users_stress["stress_score"] = users_stress["data"].apply(lambda d: d["STRESS_SCORE"])
users_stress["sleep_points_percentage"] = users_stress["data"].apply(lambda d: d["SLEEP_POINTS"]/d["MAX_SLEEP_POINTS"] if d["MAX_SLEEP_POINTS"] != 0 else d["SLEEP_POINTS"]/users_avg_max_sleep_points)
users_stress["exertion_points_percentage"] = users_stress["data"].apply(lambda d: d["EXERTION_POINTS"]/d["MAX_EXERTION_POINTS"] if d["MAX_EXERTION_POINTS"] != 0 else d["EXERTION_POINTS"]/users_avg_max_exertion_points)
users_stress["responsiveness_points_percentage"] = users_stress["data"].apply(lambda d: d["RESPONSIVENESS_POINTS"]/d["MAX_RESPONSIVENESS_POINTS"] if d["MAX_RESPONSIVENESS_POINTS"] != 0 else d["RESPONSIVENESS_POINTS"]/users_avg_max_responsiveness_points)
users_stress.drop(["data"], inplace=True, axis=1)

# convert timestamp date object and then to datetime64[ns]
users_stress["date"] = pd.to_datetime(pd.to_datetime(users_stress["date"]).dt.date)

# merge
df = df.merge(users_stress, how='outer', on=['id', 'date'])

# drop duplicates
df = df.groupby(["id", "date"]).first()
df

nightly_temperature  nremhr    rmssd  \
id                       date                                               
621e2e8e67b776a24055b564 2021-05-24            34.137687  57.432   89.603   
                         2021-05-25            33.794544  57.681   94.303   
                         2021-05-26            34.611011  57.481  119.212   
                         2021-05-27            34.408304  57.493  111.709   
                         2021-05-28            34.178922  56.750  103.034   
...                                                  ...     ...      ...   
621e375b67b776a240290cdc 2021-07-25            34.112386  65.899   22.892   
                         2021-07-26            33.895137  65.468   22.888   
                         2021-07-27            33.758319  63.808   21.189   
                         2021-07-29            34.571786  63.397   20.818   
                         2021-07-31                  NaN     NaN      NaN   

                                     spo2  full_sleep_breathing_rate  \
id                       date                                          
621e2e8e67b776a24055b564 2021-05-24   NaN                       14.8   
                         2021-05-25   NaN                       15.8   
                         2021-05-26   NaN                       14.6   
                         2021-05-27   NaN                       14.8   
                         2021-05-28   NaN                       15.2   
...                                   ...                        ...   
621e375b67b776a240290cdc 2021-07-25   NaN                       18.0   
                         2021-07-26   NaN                       17.0   
                         2021-07-27   NaN                       17.4   
                         2021-07-29   NaN                       16.8   
                         2021-07-31   NaN                        NaN   

                                     stress_score  sleep_points_percentage  \
id                       date                                                
621e2e8e67b776a24055b564 2021-05-24          78.0                 0.833333   
                         2021-05-25          80.0                 0.833333   
                         2021-05-26          84.0                 0.966667   
                         2021-05-27          82.0                 0.933333   
                         2021-05-28          81.0                 0.866667   
...                                           ...                      ...   
621e375b67b776a240290cdc 2021-07-25          84.0                 0.833333   
                         2021-07-26          89.0                 1.000000   
                         2021-07-27          82.0                 0.900000   
                         2021-07-29          84.0                 0.633333   
                         2021-07-31           0.0                 0.000000   

                                     exertion_points_percentage  \
id                       date                                     
621e2e8e67b776a24055b564 2021-05-24                       0.675   
                         2021-05-25                       0.725   
                         2021-05-26                       0.725   
                         2021-05-27                       0.725   
                         2021-05-28                       0.725   
...                                                         ...   
621e375b67b776a240290cdc 2021-07-25                       0.900   
                         2021-07-26                       0.900   
                         2021-07-27                       0.800   
                         2021-07-29                       0.900   
                         2021-07-31                       0.000   

                                     responsiveness_points_percentage  
id                       date                                          
621e2e8e67b776a24055b564 2021-05-24                          0.8666

Wrist Temperature

In [112]:
if os.path.exists('data/users_temperature.pkl'):
    print("Reading from pickle...")
    users_wrist_temperature = pd.read_pickle("data/users_temperature.pkl")
else:
    users_wrist_temperature = pd.DataFrame(columns=["id", "data"])
    for user in users:
        user_data = pd.DataFrame(list(
            db.fitbit.find({"$and": [
                {"type": "Wrist Temperature"},
                {"id": user}
            ]},
                {"id": 1, "data.recorded_time": 1, "data.temperature": 1, "_id": 0}
            )
        ))

        users_wrist_temperature = pd.concat([users_wrist_temperature, user_data], axis=0)

    # split data column (json format) into two columns
    users_wrist_temperature["date"] = users_wrist_temperature["data"].apply(lambda d: d["recorded_time"])
    users_wrist_temperature["daily_temperature_variation"] = users_wrist_temperature["data"].apply(lambda d: d["temperature"])
    users_wrist_temperature.drop(["data"], inplace=True, axis=1)

    # convert timestamp date object and then to datetime64[ns]
    users_wrist_temperature["date"] = pd.to_datetime(pd.to_datetime(users_wrist_temperature["date"]).dt.date, infer_datetime_format=True)

users_wrist_temperature.to_pickle("data/users_temperature.pkl")

# group by date and then take the average
users_wrist_temperature = users_wrist_temperature.groupby(['id', 'date']).mean()
users_wrist_temperature.reset_index(drop=False, inplace=True)

# merge
df = df.merge(users_wrist_temperature, how='outer', on=['id', 'date'])
df

Reading from pickle...


,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,daily_temperature_variation
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603,NaN,14.8,78.0,0.833333,0.675,0.866667,-1.788325
1,621e2e8e67b776a24055b564,2021-05-25,33.794544,57.681,94.303,NaN,15.8,80.0,0.833333,0.725,0.866667,-2.462709
2,621e2e8e67b776a24055b564,2021-05-26,34.611011,57.481,119.212,NaN,14.6,84.0,0.966667,0.725,0.866667,-2.385801
3,621e2e8e67b776a24055b564,2021-05-27,34.408304,57.493,111.709,NaN,14.8,82.0,0.933333,0.725,0.833333,-2.124199
4,621e2e8e67b776a24055b564,2021-05-28,34.178922,56.750,103.034,NaN,15.2,81.0,0.866667,0.725,0.866667,-2.396873
...,...,...,...,...,...,...,...,...,...,...,...,...
3530,621e375b67b776a240290cdc,2021-07-31,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,0.000,0.000000,NaN
3531,621e314867b776a24029ebf9,2021-05-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.102195
3532,621e335a67b776a240bb12ff,2021-05-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.667528
3533,621e341067b776a24037b105,2021-05-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.031670


Badge

In [113]:
users_badges = pd.DataFrame(columns=["id", "data"])
for user in users:
    user_data = pd.DataFrame(list(
        db.fitbit.find({"$and": [
            {"type": "badge"},
            {"id": user}
        ]},
            {"id": 1, "data.dateTime": 1, "data.badgeType": 1, "_id": 0}
        )
    ))

    users_badges = pd.concat([users_badges, user_data], axis=0)

# split data column (json format) into two columns
users_badges["date"] = users_badges["data"].apply(lambda d: d["dateTime"])
users_badges["badgeType"] = users_badges["data"].apply(lambda d: d["badgeType"])
users_badges.drop(["data"], inplace=True, axis=1)

# convert timestamp date object and then to datetime64[ns]
users_badges["date"] = pd.to_datetime(pd.to_datetime(users_badges["date"]).dt.date)

# group by ID and date to have only unique dates per participant
users_badges = users_badges.groupby(['id', 'date']).badgeType.apply(list).reset_index(drop=False)
users_badges.badgeType = users_badges.badgeType.apply(lambda l: list(set(l)) if isinstance(l, list) else l)
users_badges.badgeType = users_badges.badgeType.apply(lambda l: np.NaN if l == [np.nan] else l)
users_badges.badgeType.value_counts()

# merge
df = df.merge(users_badges, how='outer', on=['id', 'date'])
df

,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,daily_temperature_variation,badgeType
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603,NaN,14.8,78.0,0.833333,0.675,0.866667,-1.788325,NaN
1,621e2e8e67b776a24055b564,2021-05-25,33.794544,57.681,94.303,NaN,15.8,80.0,0.833333,0.725,0.866667,-2.462709,NaN
2,621e2e8e67b776a24055b564,2021-05-26,34.611011,57.481,119.212,NaN,14.6,84.0,0.966667,0.725,0.866667,-2.385801,NaN
3,621e2e8e67b776a24055b564,2021-05-27,34.408304,57.493,111.709,NaN,14.8,82.0,0.933333,0.725,0.833333,-2.124199,[LIFETIME_DISTANCE]
4,621e2e8e67b776a24055b564,2021-05-28,34.178922,56.750,103.034,NaN,15.2,81.0,0.866667,0.725,0.866667,-2.396873,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3673,621e375367b776a24021e950,2022-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[DAILY_STEPS]
3674,621e375367b776a24021e950,2022-01-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[LIFETIME_FLOORS]
3675,621e375367b776a24021e950,2022-01-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[DAILY_FLOORS]
3676,621e375367b776a24021e950,2022-01-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[DAILY_FLOORS]


Calories

In [114]:
if os.path.exists('data/users_calories.pkl'):
    print("Reading from pickle...")
    f = open("data/users_calories.pkl", "rb")
    # disable garbage collector
    gc.disable()
    # read pickle
    users_calories = pickle.load(f)
    # enable garbage collector again
    gc.enable()
    f.close()
else:
    users_calories = pd.DataFrame(columns=["id", "data"])
    for user in tqdm(users):
        user_data = pd.DataFrame(list(
            db.fitbit.find({"$and": [
                {"type": "calories"},
                {"id": user}
            ]},
                {"id": 1, "data.dateTime": 1, "data.value": 1, "_id": 0}
            )
        ))

        users_calories = pd.concat([users_calories, user_data], axis=0)

    print("Column Split...")
    # split data column (json format) into two columns
    users_calories.reset_index(drop=True, inplace=True)
    users_calories["date"] = users_calories["data"].apply(lambda d: d["dateTime"])
    users_calories["calories"] = users_calories["data"].apply(lambda d: d["value"])
    users_calories.drop(["data"], inplace=True, axis=1)
    print("Column Split Completed")
    print("Date Conversion...")
    # convert timestamp date object and then to datetime64[ns]
    users_calories["date"] = pd.to_datetime(users_calories["date"], format="%m/%d/%y %H:%M:%S").dt.date
    users_calories["date"] = pd.to_datetime(users_calories["date"], format="%Y/%m/%d")
    print("Date Conversion Completed")
    users_calories.to_pickle('data/users_calories.pkl')

# group by date and then take the average
users_calories.calories = users_calories.calories.astype(float)
users_calories = users_calories.groupby(['id', 'date']).sum()
users_calories.reset_index(drop=False, inplace=True)

# merge
df = df.merge(users_calories, how='outer', on=['id', 'date'])
df

Reading from pickle...


,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,daily_temperature_variation,badgeType,calories
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603,NaN,14.8,78.0,0.833333,0.675,0.866667,-1.788325,NaN,2351.59
1,621e2e8e67b776a24055b564,2021-05-25,33.794544,57.681,94.303,NaN,15.8,80.0,0.833333,0.725,0.866667,-2.462709,NaN,2332.08
2,621e2e8e67b776a24055b564,2021-05-26,34.611011,57.481,119.212,NaN,14.6,84.0,0.966667,0.725,0.866667,-2.385801,NaN,2262.30
3,621e2e8e67b776a24055b564,2021-05-27,34.408304,57.493,111.709,NaN,14.8,82.0,0.933333,0.725,0.833333,-2.124199,[LIFETIME_DISTANCE],2325.10
4,621e2e8e67b776a24055b564,2021-05-28,34.178922,56.750,103.034,NaN,15.2,81.0,0.866667,0.725,0.866667,-2.396873,NaN,2586.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6666,621e375b67b776a240290cdc,2021-08-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1296.00
6667,621e375b67b776a240290cdc,2021-08-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1296.00
6668,621e375b67b776a240290cdc,2021-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1296.00
6669,621e375b67b776a240290cdc,2021-08-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1296.00


VO2 Max

In [115]:
users_vo2max = pd.DataFrame(columns=["id", "data"])
for user in users:
    user_data = pd.DataFrame(list(
        db.fitbit.find({"$and": [
            {"type": "demographic_vo2_max"},
            {"id": user}
        ]},
            {"id": 1, "data.dateTime": 1, "data.value.filteredDemographicVO2Max": 1, "_id": 0}
        )
    ))

    users_vo2max = pd.concat([users_vo2max, user_data], axis=0)

# split data column (json format) into two columns
users_vo2max["date"] = users_vo2max["data"].apply(lambda d: d["dateTime"])
users_vo2max["filteredDemographicVO2Max"] = users_vo2max["data"].apply(lambda d: d["value"].get("filteredDemographicVO2Max"))
users_vo2max.drop(["data"], inplace=True, axis=1)

# convert timestamp date object and then to datetime64[ns]
users_vo2max["date"] = pd.to_datetime(pd.to_datetime(users_vo2max["date"]).dt.date)

# merge
df = df.merge(users_vo2max, how='outer', on=['id', 'date'])

# drop duplicates
df = df.groupby(["id", "date"]).first()
df

nightly_temperature  nremhr    rmssd  \
id                       date                                               
621e2e8e67b776a24055b564 2021-05-24            34.137687  57.432   89.603   
                         2021-05-25            33.794544  57.681   94.303   
                         2021-05-26            34.611011  57.481  119.212   
                         2021-05-27            34.408304  57.493  111.709   
                         2021-05-28            34.178922  56.750  103.034   
...                                                  ...     ...      ...   
621e375b67b776a240290cdc 2021-08-13                  NaN     NaN      NaN   
                         2021-08-14                  NaN     NaN      NaN   
                         2021-08-15                  NaN     NaN      NaN   
                         2021-08-16                  NaN     NaN      NaN   
                         2021-08-17                  NaN     NaN      NaN   

                                     spo2  full_sleep_breathing_rate  \
id                       date                                          
621e2e8e67b776a24055b564 2021-05-24   NaN                       14.8   
                         2021-05-25   NaN                       15.8   
                         2021-05-26   NaN                       14.6   
                         2021-05-27   NaN                       14.8   
                         2021-05-28   NaN                       15.2   
...                                   ...                        ...   
621e375b67b776a240290cdc 2021-08-13   NaN                        NaN   
                         2021-08-14   NaN                        NaN   
                         2021-08-15   NaN                        NaN   
                         2021-08-16   NaN                        NaN   
                         2021-08-17   NaN                        NaN   

                                     stress_score  sleep_points_percentage  \
id                       date                                                
621e2e8e67b776a24055b564 2021-05-24          78.0                 0.833333   
                         2021-05-25          80.0                 0.833333   
                         2021-05-26          84.0                 0.966667   
                         2021-05-27          82.0                 0.933333   
                         2021-05-28          81.0                 0.866667   
...                                           ...                      ...   
621e375b67b776a240290cdc 2021-08-13           NaN                      NaN   
                         2021-08-14           NaN                      NaN   
                         2021-08-15           NaN                      NaN   
                         2021-08-16           NaN                      NaN   
                         2021-08-17           NaN                      NaN   

                                     exertion_points_percentage  \
id                       date                                     
621e2e8e67b776a24055b564 2021-05-24                       0.675   
                         2021-05-25                       0.725   
                         2021-05-26                       0.725   
                         2021-05-27                       0.725   
                         2021-05-28                       0.725   
...                                                         ...   
621e375b67b776a240290cdc 2021-08-13                         NaN   
                         2021-08-14                         NaN   
                         2021-08-15                         NaN   
                         2021-08-16                         NaN   
                         2021-08-17                         NaN   

                                     responsiveness_points_percentage  \
id                       date                                           
621e2e8e67b776a24055b564 2021-05-24                          0.86

Distance

In [116]:
if os.path.exists('data/users_distance.pkl'):
    print("Reading from pickle...")
    f = open("data/users_distance.pkl", "rb")
    # disable garbage collector
    gc.disable()
    # read pickle
    users_distance = pickle.load(f)
    # enable garbage collector again
    gc.enable()
    f.close()
else:
    users_distance = pd.DataFrame(columns=["id", "data"])
    for user in tqdm(users):
        user_data = pd.DataFrame(list(
            db.fitbit.find({"$and": [
                {"type": "distance"},
                {"id": user}
            ]},
                {"id": 1, "data.dateTime": 1, "data.value": 1, "_id": 0}
            )
        ))

        users_distance = pd.concat([users_distance, user_data], axis=0)

    print("Column Split...")
    # split data column (json format) into two columns
    users_distance.reset_index(drop=True, inplace=True)
    users_distance["date"] = users_distance["data"].apply(lambda d: d["dateTime"])
    users_distance["distance"] = users_distance["data"].apply(lambda d: d["value"])
    users_distance.drop(["data"], inplace=True, axis=1)
    print("Column Split Completed")
    print("Date Conversion...")
    # convert timestamp date object and then to datetime64[ns]
    users_distance["date"] = pd.to_datetime(users_distance["date"], infer_datetime_format=True).dt.date
    users_distance["date"] = pd.to_datetime(users_distance["date"], infer_datetime_format=True)
    print("Date Conversion Completed")
    users_distance.to_pickle('data/users_distance.pkl')

# group by date and then take the average
users_distance.distance = users_distance.distance.astype(float)
users_distance = users_distance.groupby(['id', 'date']).sum()
users_distance.reset_index(drop=False, inplace=True)
users_distance.distance = users_distance.distance / 100  # converts cm to m

# merge
df = df.merge(users_distance, how='outer', on=['id', 'date'])
df

Reading from pickle...


,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,daily_temperature_variation,badgeType,calories,filteredDemographicVO2Max,distance
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603,NaN,14.8,78.0,0.833333,0.675,0.866667,-1.788325,None,2351.59,62.79210,6517.5
1,621e2e8e67b776a24055b564,2021-05-25,33.794544,57.681,94.303,NaN,15.8,80.0,0.833333,0.725,0.866667,-2.462709,None,2332.08,62.67912,7178.6
2,621e2e8e67b776a24055b564,2021-05-26,34.611011,57.481,119.212,NaN,14.6,84.0,0.966667,0.725,0.866667,-2.385801,None,2262.30,62.57307,6090.9
3,621e2e8e67b776a24055b564,2021-05-27,34.408304,57.493,111.709,NaN,14.8,82.0,0.933333,0.725,0.833333,-2.124199,[LIFETIME_DISTANCE],2325.10,62.47493,6653.1
4,621e2e8e67b776a24055b564,2021-05-28,34.178922,56.750,103.034,NaN,15.2,81.0,0.866667,0.725,0.866667,-2.396873,None,2586.76,62.41166,9557.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6666,621e375b67b776a240290cdc,2021-08-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,1296.00,NaN,NaN
6667,621e375b67b776a240290cdc,2021-08-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,1296.00,NaN,NaN
6668,621e375b67b776a240290cdc,2021-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,1296.00,NaN,NaN
6669,621e375b67b776a240290cdc,2021-08-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,1296.00,NaN,NaN


Exercise

In [117]:
users_exercise = pd.DataFrame(columns=["id", "data"])
for user in users:
    user_data = pd.DataFrame(list(
        db.fitbit.find({"$and": [
            {"type": "exercise"},
            {"id": user}
        ]},
            {"id": 1, "data.originalStartTime": 1, "data.activityTypeId": 1, "_id": 0}
        )
    ))

    users_exercise = pd.concat([users_exercise, user_data], axis=0)

# split data column (json format) into two columns
users_exercise["date"] = users_exercise["data"].apply(lambda d: d["originalStartTime"])
users_exercise["activityType"] = users_exercise["data"].apply(lambda d: d["activityTypeId"])
users_exercise.drop(["data"], inplace=True, axis=1)

# convert timestamp date object and then to datetime64[ns]
users_exercise["date"] = pd.to_datetime(pd.to_datetime(users_exercise["date"], infer_datetime_format=True).dt.date)

# Get distinct activity types
activity_types = pd.DataFrame(list(
        db.fitbit.find(
            {"type": "exercise"},
            {"id": 1, "data.activityTypeId": 1, "data.activityName": 1, "_id": 0}
        )
    ))
activity_types["activityTypeId"] = activity_types.data.apply(lambda d: d["activityTypeId"])
activity_types["activityName"] = activity_types.data.apply(lambda d: d["activityName"])
activity_types.drop(["data", "id"], inplace=True, axis=1)
activity_types = activity_types.drop_duplicates().reset_index(drop=True)
ACTIVITIES = {
    90013: "Walk",
    15000: "Sport",
    3001: "Aerobic Workout",
    52000: "Yoga/Pilates",
    90024: "Swim",
    90001: "Bike",
    20047: "Elliptical",
    2131: "Weights",
    55001: "Spinning",
    1071: "Bike",
    90009: "Run",
    20049: "Treadmill",
    53000: "Yoga/Pilates",
    55002: "Martial Arts",
    2040: "Circuit Training",
    2065: "Stairclimber",
    3000: "Workout",
    90012: "Hike",
    12339646: "Run",
    12350445: "Walk",
    23418750: "Swim",
    55003: "Bootcamp",
    15430: "Martial Arts",
    20057: "Interval Workout",
    15675: "Tennis",
    61980497: "Workout"
}
users_exercise["activityType"] = users_exercise["activityType"].apply(lambda a: ACTIVITIES.get(a))
users_exercise = users_exercise.groupby(['id', 'date']).activityType.apply(list).reset_index(drop=False)
users_exercise.activityType = users_exercise.activityType.apply(lambda l: list(set(l)) if isinstance(l, list) else l)

# merge
df = df.merge(users_exercise, how='outer', on=['id', 'date'])
df.to_pickle("data/temp_df_1.pkl")
df

,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,daily_temperature_variation,badgeType,calories,filteredDemographicVO2Max,distance,activityType
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603,NaN,14.8,78.0,0.833333,0.675,0.866667,-1.788325,None,2351.59,62.79210,6517.5,[Walk]
1,621e2e8e67b776a24055b564,2021-05-25,33.794544,57.681,94.303,NaN,15.8,80.0,0.833333,0.725,0.866667,-2.462709,None,2332.08,62.67912,7178.6,[Walk]
2,621e2e8e67b776a24055b564,2021-05-26,34.611011,57.481,119.212,NaN,14.6,84.0,0.966667,0.725,0.866667,-2.385801,None,2262.30,62.57307,6090.9,[Walk]
3,621e2e8e67b776a24055b564,2021-05-27,34.408304,57.493,111.709,NaN,14.8,82.0,0.933333,0.725,0.833333,-2.124199,[LIFETIME_DISTANCE],2325.10,62.47493,6653.1,[Walk]
4,621e2e8e67b776a24055b564,2021-05-28,34.178922,56.750,103.034,NaN,15.2,81.0,0.866667,0.725,0.866667,-2.396873,None,2586.76,62.41166,9557.9,[Walk]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6683,621e323667b776a240f19134,2021-06-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Walk]
6684,621e323667b776a240f19134,2021-06-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Walk]
6685,621e323667b776a240f19134,2021-06-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Walk]
6686,621e339967b776a240e502de,2021-10-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Spinning]


Heart Rate

In [118]:
if os.path.exists('data/temp_df_1.pkl'):
    print("Reading DataFrame from pickle...")
    f = open("data/temp_df_1.pkl", "rb")
    # disable garbage collector
    gc.disable()
    # read pickle
    df = pickle.load(f)
    # enable garbage collector again
    gc.enable()
    f.close()
    print("Reading completed.")

Reading DataFrame from pickle...
Reading completed.


In [119]:
if os.path.exists('data/users_hr_daily.pkl'):
    print("Reading daily data from pickle...")
    f = open("data/users_hr_daily.pkl", "rb")
    # disable garbage collector
    gc.disable()
    # read pickle
    users_hr = pickle.load(f)
    # enable garbage collector again
    gc.enable()
    f.close()
    print("Reading completed.")
elif os.path.exists('data/users_hr.pkl'):
    print("Reading raw data from pickle...")
    f = open("data/users_hr.pkl", "rb")
    # disable garbage collector
    gc.disable()
    # read pickle
    users_hr = pickle.load(f)
    # enable garbage collector again
    gc.enable()
    f.close()
    print("Reading completed.")

    users_hr.bpm = users_hr.bpm.astype(float)
    users_hr = users_hr.groupby(['id', 'date']).mean()
    users_hr.reset_index(drop=False, inplace=True)
else:
    warnings.warn("\nTo read and aggregate heart rate data from MongoDB you need to ensure index existence for both query (type, id) and projection (data.dateTime, data.value.bpm) fields (compound index of four fields)...\n")
    rows = 0
    users_hr = pd.DataFrame(columns=["id", "date", "bpm"])
    for user in tqdm(users):
        user_data = pd.DataFrame(list(
            db.fitbit.find({"$and": [
                {"id": user},
                {"type": "heart_rate"}
            ]},
                {"id": 1, "data.dateTime": 1, "data.value.bpm": 1, "_id": 0}
            )
        ))

        # split data column (json format) into two columns
        user_data["date"] = user_data["data"].apply(lambda d: d["dateTime"])
        user_data["bpm"] = user_data["data"].apply(lambda d: d["value"].get("bpm"))
        user_data.drop(["data"], inplace=True, axis=1)

        # convert timestamp date object and then to datetime64[ns]
        user_data["date"] = pd.to_datetime(pd.to_datetime(user_data["date"], infer_datetime_format="%Y-%m-%dT%H:%M:%S").dt.date, infer_datetime_format=True)

        # group by date and then take the average
        user_data.bpm = user_data.bpm.astype(float)
        user_data = user_data.groupby(['id', 'date']).mean()
        user_data.reset_index(drop=False, inplace=True)
        users_hr = pd.concat([users_hr, user_data], axis=0)

    users_hr.to_pickle('data/users_hr_daily.pkl')

Reading daily data from pickle...
Reading completed.


In [120]:
# convert timestamp date object and then to datetime64[ns]
users_hr["date"] = pd.to_datetime(pd.to_datetime(users_hr["date"], format="%Y/%m/%d  %H:%M:%S").dt.date, format="%Y/%m/%d")

# merge
df = df.merge(users_hr, how='outer', on=['id', 'date'])
df

,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,daily_temperature_variation,badgeType,calories,filteredDemographicVO2Max,distance,activityType,bpm
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603,NaN,14.8,78.0,0.833333,0.675,0.866667,-1.788325,None,2351.59,62.79210,6517.5,[Walk],71.701565
1,621e2e8e67b776a24055b564,2021-05-25,33.794544,57.681,94.303,NaN,15.8,80.0,0.833333,0.725,0.866667,-2.462709,None,2332.08,62.67912,7178.6,[Walk],70.5793
2,621e2e8e67b776a24055b564,2021-05-26,34.611011,57.481,119.212,NaN,14.6,84.0,0.966667,0.725,0.866667,-2.385801,None,2262.30,62.57307,6090.9,[Walk],71.842573
3,621e2e8e67b776a24055b564,2021-05-27,34.408304,57.493,111.709,NaN,14.8,82.0,0.933333,0.725,0.833333,-2.124199,[LIFETIME_DISTANCE],2325.10,62.47493,6653.1,[Walk],71.725477
4,621e2e8e67b776a24055b564,2021-05-28,34.178922,56.750,103.034,NaN,15.2,81.0,0.866667,0.725,0.866667,-2.396873,None,2586.76,62.41166,9557.9,[Walk],74.401028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6683,621e323667b776a240f19134,2021-06-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Walk],NaN
6684,621e323667b776a240f19134,2021-06-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Walk],NaN
6685,621e323667b776a240f19134,2021-06-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Walk],NaN
6686,621e339967b776a240e502de,2021-10-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[Spinning],NaN


Lightly Active Minutes

In [121]:
users_active_minutes = pd.DataFrame(columns=["id", "data"])
for user in users:
    user_data = pd.DataFrame(list(
        db.fitbit.find({"$and": [
            {"type": "lightly_active_minutes"},
            {"id": user}
        ]},
            {"id": 1, "data.dateTime": 1, "data.value": 1, "_id": 0}
        )
    ))

    users_active_minutes = pd.concat([users_active_minutes, user_data], axis=0)

# split data column (json format) into two columns
users_active_minutes["date"] = users_active_minutes["data"].apply(lambda d: d["dateTime"])
users_active_minutes["lightly_active_minutes"] = users_active_minutes["data"].apply(lambda d: d["value"])
users_active_minutes.drop(["data"], inplace=True, axis=1)

# convert timestamp date object and then to datetime64[ns]
users_active_minutes["date"] = pd.to_datetime(pd.to_datetime(users_active_minutes["date"], format="%m/%d/%y %H:%M:%S").dt.date, format="%Y/%m/%d")

# merge
df = df.merge(users_active_minutes, how='outer', on=['id', 'date'])

# drop duplicates
df = df.groupby(["id", "date"]).max()
df.reset_index(drop=False, inplace=True)
df

,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,daily_temperature_variation,badgeType,calories,filteredDemographicVO2Max,distance,activityType,bpm,lightly_active_minutes
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603,NaN,14.8,78.0,0.833333,0.675,0.866667,-1.788325,NaN,2351.59,62.79210,6517.5,[Walk],71.701565,149
1,621e2e8e67b776a24055b564,2021-05-25,33.794544,57.681,94.303,NaN,15.8,80.0,0.833333,0.725,0.866667,-2.462709,NaN,2332.08,62.67912,7178.6,[Walk],70.579300,132
2,621e2e8e67b776a24055b564,2021-05-26,34.611011,57.481,119.212,NaN,14.6,84.0,0.966667,0.725,0.866667,-2.385801,NaN,2262.30,62.57307,6090.9,[Walk],71.842573,112
3,621e2e8e67b776a24055b564,2021-05-27,34.408304,57.493,111.709,NaN,14.8,82.0,0.933333,0.725,0.833333,-2.124199,[LIFETIME_DISTANCE],2325.10,62.47493,6653.1,[Walk],71.725477,133
4,621e2e8e67b776a24055b564,2021-05-28,34.178922,56.750,103.034,NaN,15.2,81.0,0.866667,0.725,0.866667,-2.396873,NaN,2586.76,62.41166,9557.9,[Walk],74.401028,136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7106,621e375b67b776a240290cdc,2021-08-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1296.00,NaN,NaN,NaN,NaN,0
7107,621e375b67b776a240290cdc,2021-08-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1296.00,NaN,NaN,NaN,NaN,0
7108,621e375b67b776a240290cdc,2021-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1296.00,NaN,NaN,NaN,NaN,0
7109,621e375b67b776a240290cdc,2021-08-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1296.00,NaN,NaN,NaN,NaN,0


Moderately Active Minutes

In [122]:
users_active_minutes = pd.DataFrame(columns=["id", "data"])
for user in users:
    user_data = pd.DataFrame(list(
        db.fitbit.find({"$and": [
            {"type": "moderately_active_minutes"},
            {"id": user}
        ]},
            {"id": 1, "data.dateTime": 1, "data.value": 1, "_id": 0}
        )
    ))

    users_active_minutes = pd.concat([users_active_minutes, user_data], axis=0)

# split data column (json format) into two columns
users_active_minutes["date"] = users_active_minutes["data"].apply(lambda d: d["dateTime"])
users_active_minutes["moderately_active_minutes"] = users_active_minutes["data"].apply(lambda d: d["value"])
users_active_minutes.drop(["data"], inplace=True, axis=1)

# convert timestamp date object and then to datetime64[ns]
users_active_minutes["date"] = pd.to_datetime(pd.to_datetime(users_active_minutes["date"], format="%m/%d/%y %H:%M:%S").dt.date, format="%Y/%m/%d")

# merge
df = df.merge(users_active_minutes, how='outer', on=['id', 'date'])

# drop duplicates
df = df.groupby(["id", "date"]).max()
df.reset_index(drop=False, inplace=True)
df

,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,daily_temperature_variation,badgeType,calories,filteredDemographicVO2Max,distance,activityType,bpm,lightly_active_minutes,moderately_active_minutes
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603,NaN,14.8,78.0,0.833333,0.675,0.866667,-1.788325,NaN,2351.59,62.79210,6517.5,[Walk],71.701565,149,24
1,621e2e8e67b776a24055b564,2021-05-25,33.794544,57.681,94.303,NaN,15.8,80.0,0.833333,0.725,0.866667,-2.462709,NaN,2332.08,62.67912,7178.6,[Walk],70.579300,132,25
2,621e2e8e67b776a24055b564,2021-05-26,34.611011,57.481,119.212,NaN,14.6,84.0,0.966667,0.725,0.866667,-2.385801,NaN,2262.30,62.57307,6090.9,[Walk],71.842573,112,27
3,621e2e8e67b776a24055b564,2021-05-27,34.408304,57.493,111.709,NaN,14.8,82.0,0.933333,0.725,0.833333,-2.124199,[LIFETIME_DISTANCE],2325.10,62.47493,6653.1,[Walk],71.725477,133,21
4,621e2e8e67b776a24055b564,2021-05-28,34.178922,56.750,103.034,NaN,15.2,81.0,0.866667,0.725,0.866667,-2.396873,NaN,2586.76,62.41166,9557.9,[Walk],74.401028,136,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7106,621e375b67b776a240290cdc,2021-08-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1296.00,NaN,NaN,NaN,NaN,0,0
7107,621e375b67b776a240290cdc,2021-08-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1296.00,NaN,NaN,NaN,NaN,0,0
7108,621e375b67b776a240290cdc,2021-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1296.00,NaN,NaN,NaN,NaN,0,0
7109,621e375b67b776a240290cdc,2021-08-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1296.00,NaN,NaN,NaN,NaN,0,0


Very Active Minutes

In [123]:
users_active_minutes = pd.DataFrame(columns=["id", "data"])
for user in users:
    user_data = pd.DataFrame(list(
        db.fitbit.find({"$and": [
            {"type": "very_active_minutes"},
            {"id": user}
        ]},
            {"id": 1, "data.dateTime": 1, "data.value": 1, "_id": 0}
        )
    ))

    users_active_minutes = pd.concat([users_active_minutes, user_data], axis=0)

# split data column (json format) into two columns
users_active_minutes["date"] = users_active_minutes["data"].apply(lambda d: d["dateTime"])
users_active_minutes["very_active_minutes"] = users_active_minutes["data"].apply(lambda d: d["value"])
users_active_minutes.drop(["data"], inplace=True, axis=1)

# convert timestamp date object and then to datetime64[ns]
users_active_minutes["date"] = pd.to_datetime(pd.to_datetime(users_active_minutes["date"], format="%m/%d/%y %H:%M:%S").dt.date, format="%Y/%m/%d")

# merge
df = df.merge(users_active_minutes, how='outer', on=['id', 'date'])

# drop duplicates
df = df.groupby(["id", "date"]).max()
df.reset_index(drop=False, inplace=True)
df

,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,...,daily_temperature_variation,badgeType,calories,filteredDemographicVO2Max,distance,activityType,bpm,lightly_active_minutes,moderately_active_minutes,very_active_minutes
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603,NaN,14.8,78.0,0.833333,0.675,...,-1.788325,NaN,2351.59,62.79210,6517.5,[Walk],71.701565,149,24,33
1,621e2e8e67b776a24055b564,2021-05-25,33.794544,57.681,94.303,NaN,15.8,80.0,0.833333,0.725,...,-2.462709,NaN,2332.08,62.67912,7178.6,[Walk],70.579300,132,25,31
2,621e2e8e67b776a24055b564,2021-05-26,34.611011,57.481,119.212,NaN,14.6,84.0,0.966667,0.725,...,-2.385801,NaN,2262.30,62.57307,6090.9,[Walk],71.842573,112,27,31
3,621e2e8e67b776a24055b564,2021-05-27,34.408304,57.493,111.709,NaN,14.8,82.0,0.933333,0.725,...,-2.124199,[LIFETIME_DISTANCE],2325.10,62.47493,6653.1,[Walk],71.725477,133,21,37
4,621e2e8e67b776a24055b564,2021-05-28,34.178922,56.750,103.034,NaN,15.2,81.0,0.866667,0.725,...,-2.396873,NaN,2586.76,62.41166,9557.9,[Walk],74.401028,136,42,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7106,621e375b67b776a240290cdc,2021-08-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1296.00,NaN,NaN,NaN,NaN,0,0,0
7107,621e375b67b776a240290cdc,2021-08-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1296.00,NaN,NaN,NaN,NaN,0,0,0
7108,621e375b67b776a240290cdc,2021-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1296.00,NaN,NaN,NaN,NaN,0,0,0
7109,621e375b67b776a240290cdc,2021-08-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1296.00,NaN,NaN,NaN,NaN,0,0,0


Sedentary Minutes

In [124]:
users_active_minutes = pd.DataFrame(columns=["id", "data"])
for user in users:
    user_data = pd.DataFrame(list(
        db.fitbit.find({"$and": [
            {"type": "sedentary_minutes"},
            {"id": user}
        ]},
            {"id": 1, "data.dateTime": 1, "data.value": 1, "_id": 0}
        )
    ))

    users_active_minutes = pd.concat([users_active_minutes, user_data], axis=0)

# split data column (json format) into two columns
users_active_minutes["date"] = users_active_minutes["data"].apply(lambda d: d["dateTime"])
users_active_minutes["sedentary_minutes"] = users_active_minutes["data"].apply(lambda d: d["value"])
users_active_minutes.drop(["data"], inplace=True, axis=1)

# convert timestamp date object and then to datetime64[ns]
users_active_minutes["date"] = pd.to_datetime(pd.to_datetime(users_active_minutes["date"], format="%m/%d/%y %H:%M:%S").dt.date, format="%Y/%m/%d")

# merge
df = df.merge(users_active_minutes, how='outer', on=['id', 'date'])

# drop duplicates
df.to_pickle("data/temp_df_2.pkl")
df = df.groupby(["id", "date"]).max()
df.reset_index(drop=False, inplace=True)
df

,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,...,badgeType,calories,filteredDemographicVO2Max,distance,activityType,bpm,lightly_active_minutes,moderately_active_minutes,very_active_minutes,sedentary_minutes
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603,NaN,14.8,78.0,0.833333,0.675,...,NaN,2351.59,62.79210,6517.5,[Walk],71.701565,149,24,33,713
1,621e2e8e67b776a24055b564,2021-05-25,33.794544,57.681,94.303,NaN,15.8,80.0,0.833333,0.725,...,NaN,2332.08,62.67912,7178.6,[Walk],70.579300,132,25,31,704
2,621e2e8e67b776a24055b564,2021-05-26,34.611011,57.481,119.212,NaN,14.6,84.0,0.966667,0.725,...,NaN,2262.30,62.57307,6090.9,[Walk],71.842573,112,27,31,710
3,621e2e8e67b776a24055b564,2021-05-27,34.408304,57.493,111.709,NaN,14.8,82.0,0.933333,0.725,...,[LIFETIME_DISTANCE],2325.10,62.47493,6653.1,[Walk],71.725477,133,21,37,622
4,621e2e8e67b776a24055b564,2021-05-28,34.178922,56.750,103.034,NaN,15.2,81.0,0.866667,0.725,...,NaN,2586.76,62.41166,9557.9,[Walk],74.401028,136,42,54,647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7106,621e375b67b776a240290cdc,2021-08-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1296.00,NaN,NaN,NaN,NaN,0,0,0,1440
7107,621e375b67b776a240290cdc,2021-08-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1296.00,NaN,NaN,NaN,NaN,0,0,0,1440
7108,621e375b67b776a240290cdc,2021-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1296.00,NaN,NaN,NaN,NaN,0,0,0,1440
7109,621e375b67b776a240290cdc,2021-08-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1296.00,NaN,NaN,NaN,NaN,0,0,0,1440


Mindfulness Sessions

In [125]:
users_mindfulness = pd.DataFrame(columns=["id", "data"])
for user in users:
    user_data = pd.DataFrame(list(
        db.fitbit.find({"$and": [
            {"type": "mindfulness_sessions"},
            {"id": user}
        ]},
            {"id": 1, "data.start_date_time": 1, "data.session_type": 1, "_id": 0}
        )
    ))
    users_mindfulness = pd.concat([users_mindfulness, user_data], axis=0)

# split data column (json format) into two columns
users_mindfulness["date"] = users_mindfulness["data"].apply(lambda d: d["start_date_time"])
users_mindfulness["activityType"] = users_mindfulness["data"].apply(lambda d: d["session_type"])
users_mindfulness.drop(["data"], inplace=True, axis=1)

# convert timestamp date object and then to datetime64[ns]
users_mindfulness["date"] = pd.to_datetime(pd.to_datetime(users_mindfulness["date"], infer_datetime_format=True).dt.date)
users_mindfulness["mindfulness_session"] = True  # instead of storing the session type, only store if user engaged in session
users_mindfulness.drop(['activityType'], axis=1, inplace=True)
users_mindfulness = users_mindfulness.drop_duplicates()

# merge
df = df.merge(users_mindfulness, how='outer', on=['id', 'date'])
df.mindfulness_session.fillna('False', inplace=True)
df

,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,...,calories,filteredDemographicVO2Max,distance,activityType,bpm,lightly_active_minutes,moderately_active_minutes,very_active_minutes,sedentary_minutes,mindfulness_session
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603,NaN,14.8,78.0,0.833333,0.675,...,2351.59,62.79210,6517.5,[Walk],71.701565,149,24,33,713,False
1,621e2e8e67b776a24055b564,2021-05-25,33.794544,57.681,94.303,NaN,15.8,80.0,0.833333,0.725,...,2332.08,62.67912,7178.6,[Walk],70.579300,132,25,31,704,False
2,621e2e8e67b776a24055b564,2021-05-26,34.611011,57.481,119.212,NaN,14.6,84.0,0.966667,0.725,...,2262.30,62.57307,6090.9,[Walk],71.842573,112,27,31,710,False
3,621e2e8e67b776a24055b564,2021-05-27,34.408304,57.493,111.709,NaN,14.8,82.0,0.933333,0.725,...,2325.10,62.47493,6653.1,[Walk],71.725477,133,21,37,622,False
4,621e2e8e67b776a24055b564,2021-05-28,34.178922,56.750,103.034,NaN,15.2,81.0,0.866667,0.725,...,2586.76,62.41166,9557.9,[Walk],74.401028,136,42,54,647,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7106,621e375b67b776a240290cdc,2021-08-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1296.00,NaN,NaN,NaN,NaN,0,0,0,1440,False
7107,621e375b67b776a240290cdc,2021-08-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1296.00,NaN,NaN,NaN,NaN,0,0,0,1440,False
7108,621e375b67b776a240290cdc,2021-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1296.00,NaN,NaN,NaN,NaN,0,0,0,1440,False
7109,621e375b67b776a240290cdc,2021-08-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1296.00,NaN,NaN,NaN,NaN,0,0,0,1440,False


Mindfulness EDA Data Sessions

In [126]:
users_eda = pd.DataFrame(columns=["id", "data"])
for user in users:
    user_data = pd.DataFrame(list(
        db.fitbit.find({"$and": [
            {"type": "mindfulness_eda_data_sessions"},
            {"id": user}
        ]},
            {"id": 1, "data.timestamp": 1, "data.scl_avg": 1, "_id": 0}
        )
    ))

    users_eda = pd.concat([users_eda, user_data], axis=0)

# split data column (json format) into two columns
users_eda["date"] = users_eda["data"].apply(lambda d: d["timestamp"])
users_eda["scl_avg"] = users_eda["data"].apply(lambda d: d["scl_avg"])
users_eda.drop(["data"], inplace=True, axis=1)

users_eda.reset_index(drop=True, inplace=True)
# convert timestamp date object and then to datetime64[ns]
users_eda["date"] = pd.to_datetime(pd.to_datetime(users_exercise["date"], infer_datetime_format=True).dt.date)

# group by date and then take the average
users_eda = users_eda.groupby(['id', 'date']).mean()
users_eda.reset_index(drop=False, inplace=True)

# merge
df = df.merge(users_eda, how='outer', on=['id', 'date'])
df

,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,...,filteredDemographicVO2Max,distance,activityType,bpm,lightly_active_minutes,moderately_active_minutes,very_active_minutes,sedentary_minutes,mindfulness_session,scl_avg
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603,NaN,14.8,78.0,0.833333,0.675,...,62.79210,6517.5,[Walk],71.701565,149,24,33,713,False,NaN
1,621e2e8e67b776a24055b564,2021-05-25,33.794544,57.681,94.303,NaN,15.8,80.0,0.833333,0.725,...,62.67912,7178.6,[Walk],70.579300,132,25,31,704,False,NaN
2,621e2e8e67b776a24055b564,2021-05-26,34.611011,57.481,119.212,NaN,14.6,84.0,0.966667,0.725,...,62.57307,6090.9,[Walk],71.842573,112,27,31,710,False,NaN
3,621e2e8e67b776a24055b564,2021-05-27,34.408304,57.493,111.709,NaN,14.8,82.0,0.933333,0.725,...,62.47493,6653.1,[Walk],71.725477,133,21,37,622,False,NaN
4,621e2e8e67b776a24055b564,2021-05-28,34.178922,56.750,103.034,NaN,15.2,81.0,0.866667,0.725,...,62.41166,9557.9,[Walk],74.401028,136,42,54,647,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7348,621e2f9167b776a240011ccb,2021-09-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.179268
7349,621e2f9167b776a240011ccb,2021-09-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.343615
7350,621e2f9167b776a240011ccb,2021-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.323018
7351,621e2f9167b776a240011ccb,2021-10-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.019217


Resting Heart Rate

In [127]:
users_rhr = pd.DataFrame(columns=["id", "data"])
for user in users:
    user_data = pd.DataFrame(list(
        db.fitbit.find({"$and": [
            {"type": "resting_heart_rate"},
            {"id": user}
        ]},
            {"id": 1, "data.value.date": 1, "data.value.value": 1, "_id": 0}
        )
    ))

    users_rhr = pd.concat([users_rhr, user_data], axis=0)

# split data column (json format) into two columns
users_rhr["date"] = users_rhr["data"].apply(lambda d: d["value"].get("date"))
users_rhr["resting_hr"] = users_rhr["data"].apply(lambda d: d["value"].get("value"))
users_rhr.drop(["data"], inplace=True, axis=1)

# convert timestamp date object and then to datetime64[ns]
users_rhr["date"] = pd.to_datetime(users_rhr["date"], format="%m/%d/%y")
users_rhr = users_rhr[users_rhr.resting_hr != 0.0]

# merge
df = df.merge(users_rhr, how='outer', on=['id', 'date'])

# drop duplicates
df = df.groupby(["id", "date"]).first()
df.reset_index(drop=False, inplace=True)
df

,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,...,distance,activityType,bpm,lightly_active_minutes,moderately_active_minutes,very_active_minutes,sedentary_minutes,mindfulness_session,scl_avg,resting_hr
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603,NaN,14.8,78.0,0.833333,0.675,...,6517.5,[Walk],71.701565,149,24,33,713,False,NaN,62.073070
1,621e2e8e67b776a24055b564,2021-05-25,33.794544,57.681,94.303,NaN,15.8,80.0,0.833333,0.725,...,7178.6,[Walk],70.579300,132,25,31,704,False,NaN,62.121476
2,621e2e8e67b776a24055b564,2021-05-26,34.611011,57.481,119.212,NaN,14.6,84.0,0.966667,0.725,...,6090.9,[Walk],71.842573,112,27,31,710,False,NaN,62.263999
3,621e2e8e67b776a24055b564,2021-05-27,34.408304,57.493,111.709,NaN,14.8,82.0,0.933333,0.725,...,6653.1,[Walk],71.725477,133,21,37,622,False,NaN,62.368900
4,621e2e8e67b776a24055b564,2021-05-28,34.178922,56.750,103.034,NaN,15.2,81.0,0.866667,0.725,...,9557.9,[Walk],74.401028,136,42,54,647,False,NaN,61.965409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7348,621e375b67b776a240290cdc,2021-08-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,None,NaN,0,0,0,1440,False,NaN,NaN
7349,621e375b67b776a240290cdc,2021-08-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,None,NaN,0,0,0,1440,False,NaN,NaN
7350,621e375b67b776a240290cdc,2021-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,None,NaN,0,0,0,1440,False,NaN,NaN
7351,621e375b67b776a240290cdc,2021-08-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,None,NaN,0,0,0,1440,False,NaN,NaN


Sleep

In [128]:
def try_sleep_minutes(d, type):
    try:
        ratio = d["levels"].get("summary").get(type).get("minutes")/d["levels"].get("summary").get(type).get("thirtyDayAvgMinutes")
    except AttributeError:
        ratio = np.nan
    except ZeroDivisionError:
        ratio = np.nan

    return ratio

users_sleep = pd.DataFrame(columns=["id", "data"])
for user in users:
    user_data = pd.DataFrame(list(
        db.fitbit.find({"$and": [
            {"type": "sleep"},
            {"id": user},
            {"data.mainSleep": True}
        ]},
            {"id": 1, "data.dateOfSleep": 1, "data.duration": 1, "data.minutesToFallAsleep": 1, "data.minutesAsleep": 1, "data.minutesAwake": 1, "data.minutesAfterWakeup": 1, "data.efficiency": 1, "data.levels.summary.deep.minutes": 1, "data.levels.summary.deep.thirtyDayAvgMinutes": 1, "data.levels.summary.wake.minutes": 1, "data.levels.summary.wake.thirtyDayAvgMinutes": 1, "data.levels.summary.light.minutes": 1, "data.levels.summary.light.thirtyDayAvgMinutes": 1, "data.levels.summary.rem.minutes": 1, "data.levels.summary.rem.thirtyDayAvgMinutes": 1, "_id": 0}
        )
    ))

    users_sleep = pd.concat([users_sleep, user_data], axis=0)

# split data column (json format) into two columns
users_sleep["date"] = users_sleep["data"].apply(lambda d: d["dateOfSleep"])
users_sleep["sleep_duration"] = users_sleep["data"].apply(lambda d: d["duration"])
users_sleep["minutesToFallAsleep"] = users_sleep["data"].apply(lambda d: d["minutesToFallAsleep"])
users_sleep["minutesAsleep"] = users_sleep["data"].apply(lambda d: d["minutesAsleep"])
users_sleep["minutesAwake"] = users_sleep["data"].apply(lambda d: d["minutesAwake"])
users_sleep["minutesAfterWakeup"] = users_sleep["data"].apply(lambda d: d["minutesAfterWakeup"])
users_sleep["sleep_efficiency"] = users_sleep["data"].apply(lambda d: d["efficiency"])
users_sleep["sleep_deep_ratio"] = users_sleep["data"].apply(lambda d: try_sleep_minutes(d, "deep"))
users_sleep["sleep_wake_ratio"] = users_sleep["data"].apply(lambda d: try_sleep_minutes(d, "wake"))
users_sleep["sleep_light_ratio"] = users_sleep["data"].apply(lambda d: try_sleep_minutes(d, "light"))
users_sleep["sleep_rem_ratio"] = users_sleep["data"].apply(lambda d: try_sleep_minutes(d, "rem"))
users_sleep.drop(["data"], inplace=True, axis=1)

# convert timestamp date object and then to datetime64[ns]
users_sleep["date"] = pd.to_datetime(users_sleep["date"], infer_datetime_format=True)

# merge
df = df.merge(users_sleep, how='outer', on=['id', 'date'])

# drop duplicates
df = df.groupby(["id", "date"]).first()
df.reset_index(drop=False, inplace=True)
df.to_pickle('./data/temp_df_3.pkl')
df

,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,...,sleep_duration,minutesToFallAsleep,minutesAsleep,minutesAwake,minutesAfterWakeup,sleep_efficiency,sleep_deep_ratio,sleep_wake_ratio,sleep_light_ratio,sleep_rem_ratio
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603,NaN,14.8,78.0,0.833333,0.675,...,31260000.0,0.0,445.0,76.0,0.0,93.0,1.243243,0.987013,0.921642,1.341772
1,621e2e8e67b776a24055b564,2021-05-25,33.794544,57.681,94.303,NaN,15.8,80.0,0.833333,0.725,...,32880000.0,0.0,460.0,88.0,0.0,94.0,1.466667,1.142857,0.947566,1.197531
2,621e2e8e67b776a24055b564,2021-05-26,34.611011,57.481,119.212,NaN,14.6,84.0,0.966667,0.725,...,33600000.0,0.0,493.0,67.0,0.0,96.0,1.116883,0.858974,1.015038,1.670732
3,621e2e8e67b776a24055b564,2021-05-27,34.408304,57.493,111.709,NaN,14.8,82.0,0.933333,0.725,...,37620000.0,0.0,540.0,87.0,0.0,93.0,1.128205,1.129870,1.191729,1.588235
4,621e2e8e67b776a24055b564,2021-05-28,34.178922,56.750,103.034,NaN,15.2,81.0,0.866667,0.725,...,33660000.0,0.0,493.0,68.0,0.0,94.0,0.910256,0.871795,1.211896,1.090909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7348,621e375b67b776a240290cdc,2021-08-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7349,621e375b67b776a240290cdc,2021-08-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7350,621e375b67b776a240290cdc,2021-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7351,621e375b67b776a240290cdc,2021-08-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Steps

In [129]:
if not os.path.exists("data/users_steps_daily.pkl"):
    steps_dataframe = pd.DataFrame(columns=["date", "steps", "id"])
    for user in tqdm(users):
        user_dataframe = pd.DataFrame(list(
            db.fitbit.find(
                {"type": "steps",
                 "id": user},
                {"data.dateTime": 1, "data.value": 1, "id": 1, "_id": 0}
            )
        ))
        user_dataframe['date'] = user_dataframe['data'].apply(lambda d: d['dateTime'])
        user_dataframe['steps'] = user_dataframe['data'].apply(lambda d: d['value'])
        user_dataframe.drop(["data"], inplace=True, axis=1)

        # basic preprocessing for steps - transformations
        user_dataframe['steps'] = pd.to_numeric(user_dataframe['steps'])  # was string
        user_dataframe["date"] = pd.to_datetime(pd.to_datetime(user_dataframe["date"], infer_datetime_format=True).dt.date, infer_datetime_format=True)

        steps_dataframe = pd.concat([steps_dataframe, user_dataframe], axis=0, ignore_index=True)

    # group by date and then take the sum
    steps_dataframe = steps_dataframe.groupby(['id', 'date']).sum()
    steps_dataframe.reset_index(drop=False, inplace=True)
    steps_dataframe.to_pickle("data/users_steps_daily.pkl")

infile = open('data/users_steps_daily.pkl','rb')
steps_daily = pickle.load(infile)
infile.close()

# merge
df = df.merge(steps_daily, how='outer', on=['id', 'date'])
df

,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,...,minutesToFallAsleep,minutesAsleep,minutesAwake,minutesAfterWakeup,sleep_efficiency,sleep_deep_ratio,sleep_wake_ratio,sleep_light_ratio,sleep_rem_ratio,steps
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603,NaN,14.8,78.0,0.833333,0.675,...,0.0,445.0,76.0,0.0,93.0,1.243243,0.987013,0.921642,1.341772,8833.0
1,621e2e8e67b776a24055b564,2021-05-25,33.794544,57.681,94.303,NaN,15.8,80.0,0.833333,0.725,...,0.0,460.0,88.0,0.0,94.0,1.466667,1.142857,0.947566,1.197531,9727.0
2,621e2e8e67b776a24055b564,2021-05-26,34.611011,57.481,119.212,NaN,14.6,84.0,0.966667,0.725,...,0.0,493.0,67.0,0.0,96.0,1.116883,0.858974,1.015038,1.670732,8253.0
3,621e2e8e67b776a24055b564,2021-05-27,34.408304,57.493,111.709,NaN,14.8,82.0,0.933333,0.725,...,0.0,540.0,87.0,0.0,93.0,1.128205,1.129870,1.191729,1.588235,9015.0
4,621e2e8e67b776a24055b564,2021-05-28,34.178922,56.750,103.034,NaN,15.2,81.0,0.866667,0.725,...,0.0,493.0,68.0,0.0,94.0,0.910256,0.871795,1.211896,1.090909,12949.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7348,621e375b67b776a240290cdc,2021-08-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7349,621e375b67b776a240290cdc,2021-08-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7350,621e375b67b776a240290cdc,2021-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7351,621e375b67b776a240290cdc,2021-08-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Time in Heart Rate Zones

In [130]:
users_time_in_heart_rate_zones = pd.DataFrame(columns=["id", "data"])
for user in users:
    user_data = pd.DataFrame(list(
        db.fitbit.find({"$and": [
            {"type": "time_in_heart_rate_zones"},
            {"id": user}
        ]},
            {"id": 1, "data.dateTime": 1, "data.value": 1, "_id": 0}
        )
    ))

    users_time_in_heart_rate_zones = pd.concat([users_time_in_heart_rate_zones, user_data], axis=0)

# split data column (json format) into two columns
users_time_in_heart_rate_zones["date"] = users_time_in_heart_rate_zones["data"].apply(lambda d: d["dateTime"])
users_time_in_heart_rate_zones["minutes_in_default_zone_1"] = users_time_in_heart_rate_zones["data"].apply(lambda d: d["value"].get("valuesInZones").get("IN_DEFAULT_ZONE_1"))
users_time_in_heart_rate_zones["minutes_below_default_zone_1"] = users_time_in_heart_rate_zones["data"].apply(lambda d: d["value"].get("valuesInZones").get("BELOW_DEFAULT_ZONE_1"))
users_time_in_heart_rate_zones["minutes_in_default_zone_2"] = users_time_in_heart_rate_zones["data"].apply(lambda d: d["value"].get("valuesInZones").get("IN_DEFAULT_ZONE_2"))
users_time_in_heart_rate_zones["minutes_in_default_zone_3"] = users_time_in_heart_rate_zones["data"].apply(lambda d: d["value"].get("valuesInZones").get("IN_DEFAULT_ZONE_3"))
users_time_in_heart_rate_zones.drop(["data"], inplace=True, axis=1)

# convert timestamp date object and then to datetime64[ns]
users_time_in_heart_rate_zones["date"] = pd.to_datetime(pd.to_datetime(users_time_in_heart_rate_zones["date"], format="%m/%d/%y %H:%M:%S").dt.date, format="%Y/%m/%d")

# merge
df = df.merge(users_time_in_heart_rate_zones, how='outer', on=['id', 'date'])
df

,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,...,sleep_efficiency,sleep_deep_ratio,sleep_wake_ratio,sleep_light_ratio,sleep_rem_ratio,steps,minutes_in_default_zone_1,minutes_below_default_zone_1,minutes_in_default_zone_2,minutes_in_default_zone_3
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603,NaN,14.8,78.0,0.833333,0.675,...,93.0,1.243243,0.987013,0.921642,1.341772,8833.0,83.0,1349.0,0.0,0.0
1,621e2e8e67b776a24055b564,2021-05-25,33.794544,57.681,94.303,NaN,15.8,80.0,0.833333,0.725,...,94.0,1.466667,1.142857,0.947566,1.197531,9727.0,56.0,1374.0,4.0,0.0
2,621e2e8e67b776a24055b564,2021-05-26,34.611011,57.481,119.212,NaN,14.6,84.0,0.966667,0.725,...,96.0,1.116883,0.858974,1.015038,1.670732,8253.0,85.0,1350.0,0.0,0.0
3,621e2e8e67b776a24055b564,2021-05-27,34.408304,57.493,111.709,NaN,14.8,82.0,0.933333,0.725,...,93.0,1.128205,1.129870,1.191729,1.588235,9015.0,90.0,1282.0,0.0,0.0
4,621e2e8e67b776a24055b564,2021-05-28,34.178922,56.750,103.034,NaN,15.2,81.0,0.866667,0.725,...,94.0,0.910256,0.871795,1.211896,1.090909,12949.0,146.0,1274.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7416,621e375b67b776a240290cdc,2021-08-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7417,621e375b67b776a240290cdc,2021-08-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7418,621e375b67b776a240290cdc,2021-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7419,621e375b67b776a240290cdc,2021-08-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Profile

In [131]:
def get_age(date_of_birth):
    today = date.today()
    return today.year - date_of_birth.year - ((today.month, today.day) < (date_of_birth.month, date_of_birth.day))

In [132]:
users_profiles = pd.DataFrame(columns=["id", "data"])
for user in users:
    user_data = pd.DataFrame(list(
        db.fitbit.find({"$and": [
            {"type": "Profile"},
            {"id": user}
        ]},
            {"id": 1, "data.age": 1, "data.gender": 1, "data.bmi": 1,  "_id": 0}
        )
    ))

    users_profiles = pd.concat([users_profiles, user_data], axis=0)

users_profiles["age"] = users_profiles["data"].apply(lambda d: d["age"] if "age" in d else np.NaN)
users_profiles["gender"] = users_profiles["data"].apply(lambda d: d["gender"] if "gender" in d else np.NaN)
users_profiles["bmi"] = users_profiles["data"].apply(lambda d: d["bmi"] if "bmi" in d else np.NaN)
users_profiles.drop(['data'], axis=1, inplace=True)

# merge
df = df.merge(users_profiles, how='left', on=['id'])
df.to_pickle('./data/daily_fitbit_df_unprocessed.pkl')
df = pd.read_pickle('./data/daily_fitbit_df_unprocessed.pkl')
df

,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,...,sleep_light_ratio,sleep_rem_ratio,steps,minutes_in_default_zone_1,minutes_below_default_zone_1,minutes_in_default_zone_2,minutes_in_default_zone_3,age,gender,bmi
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603,NaN,14.8,78.0,0.833333,0.675,...,0.921642,1.341772,8833.0,83.0,1349.0,0.0,0.0,<30,MALE,<19
1,621e2e8e67b776a24055b564,2021-05-25,33.794544,57.681,94.303,NaN,15.8,80.0,0.833333,0.725,...,0.947566,1.197531,9727.0,56.0,1374.0,4.0,0.0,<30,MALE,<19
2,621e2e8e67b776a24055b564,2021-05-26,34.611011,57.481,119.212,NaN,14.6,84.0,0.966667,0.725,...,1.015038,1.670732,8253.0,85.0,1350.0,0.0,0.0,<30,MALE,<19
3,621e2e8e67b776a24055b564,2021-05-27,34.408304,57.493,111.709,NaN,14.8,82.0,0.933333,0.725,...,1.191729,1.588235,9015.0,90.0,1282.0,0.0,0.0,<30,MALE,<19
4,621e2e8e67b776a24055b564,2021-05-28,34.178922,56.750,103.034,NaN,15.2,81.0,0.866667,0.725,...,1.211896,1.090909,12949.0,146.0,1274.0,4.0,0.0,<30,MALE,<19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7416,621e375b67b776a240290cdc,2021-08-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<30,FEMALE,23.0
7417,621e375b67b776a240290cdc,2021-08-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<30,FEMALE,23.0
7418,621e375b67b776a240290cdc,2021-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<30,FEMALE,23.0
7419,621e375b67b776a240290cdc,2021-08-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<30,FEMALE,23.0


Integrating SEMA Data

Integrating step goals

In [133]:
# user goals in SEMA
SEMA_GOALS_TO_MIN = {
    "2000": 0,
    "4999": 2000,
    "7999": 5000,
    "9999": 8000,
    "14999": 10000,
    "19999": 15000,
    "24999": 20000,
    "25000": 25000,
    "NO_GOAL": 0
}

SEMA_GOALS_TO_MAX = {
    "2000": 2000,
    "4999": 5000,
    "7999": 8000,
    "9999": 10000,
    "14999": 15000,
    "19999": 20000,
    "24999": 25000,
    "25000": 30000,
    "NO_GOAL": 0
}

SEMA_LABELS = {
    "2000": "Less than 2000",
    "4999": "2000-4999",
    "7999": "5000-7999",
    "9999": "8000-9999",
    "14999": "10000-14999",
    "19999": "15000-19999",
    "24999": "20000-24999",
    "25000": "More than 25000",
    "NO_GOAL": "No Goal",
    np.nan: "No Goal",
    None: "No Goal"
}

In [134]:
if not os.path.exists("data/users_step_goals_daily.pkl"):
    # Get user self-reported goals from SEMA
    sema_goals = pd.DataFrame(columns=["_id", "user_id", "data"])
    for user in users:
        user_data = pd.DataFrame(list(
                        db.sema.find({ "$and": [
                            { "data.STEPS": { "$ne": "<no-response>" } },
                            {"user_id": user}
                        ] },
                            {"data.STEPS": 1, "id": 1, "user_id": 1, "data.STARTED_TS": 1}
                        )
                    ))

        sema_goals = pd.concat([sema_goals, user_data], axis=0)

    # split data column (json format) into two columns
    sema_goals["timestamp"] = sema_goals["data"].apply(lambda d: d["STARTED_TS"])
    sema_goals["step_goal"] = sema_goals["data"].apply(lambda d: d["STEPS"])
    sema_goals.drop(["data", "_id"], inplace=True, axis=1)

    # convert timestamp to day format
    sema_goals["date"] = pd.to_datetime(sema_goals["timestamp"], infer_datetime_format=True).dt.date
    sema_goals["date"] = pd.to_datetime(sema_goals["date"], infer_datetime_format=True) # convert from object to datetime
    sema_goals.drop(["timestamp"], inplace=True, axis=1)

    # add min goal and max goal columns
    sema_goals['min_goal'] = sema_goals.step_goal.apply(lambda s: SEMA_GOALS_TO_MIN.get(s))
    sema_goals['max_goal'] = sema_goals.step_goal.apply(lambda s: SEMA_GOALS_TO_MAX.get(s))

    # add goal labels
    sema_goals['step_goal_label'] = sema_goals['step_goal'].apply(lambda v: SEMA_LABELS[v])
    sema_goals[['date', 'user_id', 'step_goal', 'min_goal', 'max_goal', 'step_goal_label']].to_pickle('./data/users_step_goals_daily.pkl')

users_step_goals = pd.read_pickle('./data/users_step_goals_daily.pkl')
users_step_goals['id'] = users_step_goals.user_id.copy()
users_step_goals.drop(['user_id'], axis=1, inplace=True)

# merge
df = df.merge(users_step_goals, how='outer', on=['id', 'date'])

# drop duplicates
df = df.groupby(["id", "date"]).first()
df.reset_index(drop=False, inplace=True)
df

,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,...,minutes_below_default_zone_1,minutes_in_default_zone_2,minutes_in_default_zone_3,age,gender,bmi,step_goal,min_goal,max_goal,step_goal_label
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603,NaN,14.8,78.0,0.833333,0.675,...,1349.0,0.0,0.0,<30,MALE,<19,None,NaN,NaN,None
1,621e2e8e67b776a24055b564,2021-05-25,33.794544,57.681,94.303,NaN,15.8,80.0,0.833333,0.725,...,1374.0,4.0,0.0,<30,MALE,<19,14999,10000.0,15000.0,10000-14999
2,621e2e8e67b776a24055b564,2021-05-26,34.611011,57.481,119.212,NaN,14.6,84.0,0.966667,0.725,...,1350.0,0.0,0.0,<30,MALE,<19,None,NaN,NaN,None
3,621e2e8e67b776a24055b564,2021-05-27,34.408304,57.493,111.709,NaN,14.8,82.0,0.933333,0.725,...,1282.0,0.0,0.0,<30,MALE,<19,14999,10000.0,15000.0,10000-14999
4,621e2e8e67b776a24055b564,2021-05-28,34.178922,56.750,103.034,NaN,15.2,81.0,0.866667,0.725,...,1274.0,4.0,0.0,<30,MALE,<19,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7395,621e375b67b776a240290cdc,2021-08-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,<30,FEMALE,23.0,None,NaN,NaN,None
7396,621e375b67b776a240290cdc,2021-08-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,<30,FEMALE,23.0,None,NaN,NaN,None
7397,621e375b67b776a240290cdc,2021-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,<30,FEMALE,23.0,None,NaN,NaN,None
7398,621e375b67b776a240290cdc,2021-08-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,<30,FEMALE,23.0,None,NaN,NaN,None


Integrating Emotions & Location

In [135]:
if not os.path.exists('./data/sema_mood_place.pkl'):
    # Get user self-reported goals from SEMA
    users = db.sema.distinct('user_id')

    sema_mood = pd.DataFrame(columns=["_id", "user_id", "data"])
    for user in users:
        user_data = pd.DataFrame(list(
                        db.sema.find({
                            "$or": [
                                {
                                    "$and": [
                                        { "data.MOOD": { "$ne": "<no-response>" } },
                                        {"data.MOOD": { "$ne": None }},
                                        {"user_id": user}
                                    ]
                                },
                                {
                                    "$and": [
                                        { "data.PLACE": { "$ne": "<no-response>" } },
                                        {"data.PLACE": { "$ne": None }},
                                        {"user_id": user}
                                    ]
                                }
                            ]
                        },
                            {"data.MOOD": 1, "data.PLACE": 1, "id": 1, "_id": 0, "user_id": 1, "data.STARTED_TS": 1}
                        )
                    ))

        sema_mood = pd.concat([sema_mood, user_data], axis=0)

    sema_mood["date"] = pd.to_datetime(pd.to_datetime(sema_mood["data"].apply(lambda d: d["STARTED_TS"]), infer_datetime_format=True).dt.date, infer_datetime_format=True)
    sema_mood["time"] = pd.to_datetime(sema_mood["data"].apply(lambda d: d["STARTED_TS"]), infer_datetime_format=True).dt.time
    sema_mood["data.MOOD"] = sema_mood["data"].apply(lambda d: d["MOOD"])
    sema_mood["data.PLACE"] = sema_mood["data"].apply(lambda d: d["PLACE"])
    sema_mood.drop(["_id", "data"], axis=1, inplace=True)
    sema_mood.to_pickle('./data/sema_mood_place.pkl')

In [136]:
infile = open('./data/sema_mood_place.pkl','rb')
sema = pickle.load(infile)
infile.close()

# replace not common moods
sema['data.MOOD'] = sema['data.MOOD'].apply(lambda mood: 'SAD' if mood == 'SADNESS' else ('HAPPY' if mood == 'JOY' else mood))
sema = sema[(sema['data.MOOD'] != 'FEAR') & (sema['data.MOOD'] != 'SURPRISE') & (sema['data.MOOD'] != 'ANGER') & (sema['data.MOOD'] != '<no-response>')]

# drop unnecessary columns
sema.drop(['time'], axis=1, inplace=True)

sema_moods = pd.get_dummies(sema['data.MOOD'])
sema_places = pd.get_dummies(sema['data.PLACE'])

# combine one-hot encoding with actual df
sema = pd.concat([sema, sema_moods, sema_places], axis=1)
sema.drop(['data.MOOD', 'data.PLACE'], axis=1, inplace=True)

sema_grouped = sema.groupby(['date', 'user_id']).max()
sema_grouped.reset_index(drop=False, inplace=True)
sema_grouped['id'] = sema_grouped['user_id'].copy()
sema_grouped.drop(['user_id'], axis=1, inplace=True)
sema_grouped.reset_index(drop=True, inplace=True)
sema_grouped.id = sema_grouped.id.apply(lambda id: ObjectId(id))

# merge
df = df.merge(sema_grouped, how='outer', on=['id', 'date'])
df.to_pickle('./data/daily_fitbit_sema_df_unprocessed.pkl')
df = pd.read_pickle('./data/daily_fitbit_sema_df_unprocessed.pkl')
df.to_csv('./data/daily_fitbit_sema_df_unprocessed.csv')
df

,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,...,TENSE/ANXIOUS,TIRED,ENTERTAINMENT,GYM,HOME,HOME_OFFICE,OTHER,OUTDOORS,TRANSIT,WORK/SCHOOL
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603,NaN,14.8,78.0,0.833333,0.675,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,621e2e8e67b776a24055b564,2021-05-25,33.794544,57.681,94.303,NaN,15.8,80.0,0.833333,0.725,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,621e2e8e67b776a24055b564,2021-05-26,34.611011,57.481,119.212,NaN,14.6,84.0,0.966667,0.725,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,621e2e8e67b776a24055b564,2021-05-27,34.408304,57.493,111.709,NaN,14.8,82.0,0.933333,0.725,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
4,621e2e8e67b776a24055b564,2021-05-28,34.178922,56.750,103.034,NaN,15.2,81.0,0.866667,0.725,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7405,621e362467b776a2404ad513,2021-05-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7406,621e36f967b776a240e5e7c9,2021-05-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
7407,621e362467b776a2404ad513,2021-05-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7408,621e339967b776a240e502de,2021-11-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


Integrating Surveys Data

In [137]:
df = pd.read_pickle('./data/daily_fitbit_sema_df_unprocessed.pkl')

Integrating Personality Type Responses

In [138]:
users_personality = pd.DataFrame(columns=["user_id", "data"])
for user in users:
    user_data = pd.DataFrame(list(
        db.surveys.find({"$and": [
            {"type": "bfpt"},
            {"user_id": user}
        ]},
            {"_id": 0}
        )
    ))

    users_personality = pd.concat([users_personality, user_data], axis=0)

users_personality["submitdate"] = users_personality["data"].apply(lambda d: d["submitdate"])
users_personality["submitdate"] = pd.to_datetime(users_personality["submitdate"], infer_datetime_format=True).dt.date

for ipip in range(1,51):
    if ipip < 10:
        col_name = "ipip[SQ00{}]".format(ipip)
    else:
        col_name = "ipip[SQ0{}]".format(ipip)
    users_personality[col_name] = users_personality["data"].apply(lambda d: d[col_name])

users_personality.drop(['data'], axis=1, inplace=True)

# drop duplicates
users_personality.drop_duplicates(subset=['user_id'], keep="first")

bfpt_scoring = pd.read_csv("data/utils/BFPT-Coding.csv", sep=";")
ipip_plus = bfpt_scoring[bfpt_scoring.plus == True].code
ipip_minus = bfpt_scoring[bfpt_scoring.plus == False].code

# Converting IPIP Item Responses to Scale Scores
# For + keyed items, the response "Very Inaccurate" is assigned a value of 1, "Moderately Inaccurate" a value of 2, "Neither Inaccurate nor Accurate" a 3, "Moderately Accurate" a 4, and "Very Accurate" a value of 5.
# For - keyed items, the response "Very Inaccurate" is assigned a value of 5, "Moderately Inaccurate" a value of 4, "Neither Inaccurate nor Accurate" a 3, "Moderately Accurate" a 2, and "Very Accurate" a value of 1.
def inverse_score(score, min, max):
    return max - score + min

for col in users_personality.columns:
    # inversing scores for the minus keyed items; the plus keyed items stay as is
    if col in ipip_minus.values:
        users_personality[col] = users_personality[col].apply(lambda score: inverse_score(score, 1, 5))

ipip_extraversion = bfpt_scoring[bfpt_scoring.factor == 1].code
ipip_agreeableness = bfpt_scoring[bfpt_scoring.factor == 2].code
ipip_conscientiousness = bfpt_scoring[bfpt_scoring.factor == 3].code
ipip_stability = bfpt_scoring[bfpt_scoring.factor == 4].code
ipip_intellect = bfpt_scoring[bfpt_scoring.factor == 5].code

# find a summary per factor per user
# Factor I (Surgency or Extraversion)
users_personality["extraversion"] = users_personality[ipip_extraversion].sum(axis=1)
# Factor II (Agreeableness)
users_personality["agreeableness"] = users_personality[ipip_agreeableness].sum(axis=1)
# Factor III (Conscientiousness)
users_personality["conscientiousness"] = users_personality[ipip_conscientiousness].sum(axis=1)
# Factor IV (Emotional Stability)
users_personality["stability"] = users_personality[ipip_stability].sum(axis=1)
# Factor V (Intellect or Imagination)
users_personality["intellect"] = users_personality[ipip_intellect].sum(axis=1)

In [139]:
# incorporate gender information
users_personality = users_personality.merge(df[['id','gender']].drop_duplicates(), how='left', left_on='user_id', right_on='id')
users_personality.drop_duplicates(subset='id', inplace=True)
users_personality.gender.fillna('FEMALE', inplace=True)
users_personality.drop(['id'], inplace=True, axis=1)

In [140]:
MEAN_1_FEMALE, STD_1_FEMALE = users_personality.groupby('gender').extraversion.mean()["FEMALE"], users_personality.groupby('gender').extraversion.std()["FEMALE"]
MEAN_1_MALE, STD_1_MALE = users_personality.groupby('gender').extraversion.mean()["MALE"], users_personality.groupby('gender').extraversion.std()["MALE"]

MEAN_2_FEMALE, STD_2_FEMALE = users_personality.groupby('gender').agreeableness.mean()["FEMALE"], users_personality.groupby('gender').agreeableness.std()["FEMALE"]
MEAN_2_MALE, STD_2_MALE = users_personality.groupby('gender').agreeableness.mean()["MALE"], users_personality.groupby('gender').agreeableness.std()["MALE"]

MEAN_3_FEMALE, STD_3_FEMALE = users_personality.groupby('gender').conscientiousness.mean()["FEMALE"], users_personality.groupby('gender').conscientiousness.std()["FEMALE"]
MEAN_3_MALE, STD_3_MALE = users_personality.groupby('gender').conscientiousness.mean()["MALE"], users_personality.groupby('gender').conscientiousness.std()["MALE"]

MEAN_4_FEMALE, STD_4_FEMALE = users_personality.groupby('gender').stability.mean()["FEMALE"], users_personality.groupby('gender').stability.std()["FEMALE"]
MEAN_4_MALE, STD_4_MALE = users_personality.groupby('gender').stability.mean()["MALE"], users_personality.groupby('gender').stability.std()["MALE"]

MEAN_5_FEMALE, STD_5_FEMALE = users_personality.groupby('gender').intellect.mean()["FEMALE"], users_personality.groupby('gender').intellect.std()["FEMALE"]
MEAN_5_MALE, STD_5_MALE = users_personality.groupby('gender').intellect.mean()["MALE"], users_personality.groupby('gender').intellect.std()["MALE"]

In [141]:
def get_personality_category(score, mean, std):
    if score > mean + 0.5*std:
        return 'HIGH'
    if score < mean - 0.5*std:
        return 'LOW'
    return 'AVERAGE'

users_personality['ipip_extraversion_category'] = users_personality.apply(lambda row: get_personality_category(row.extraversion, MEAN_1_MALE, STD_1_MALE) if row.gender == "MALE" else get_personality_category(row.extraversion, MEAN_1_FEMALE, STD_1_FEMALE), axis=1)

users_personality['ipip_agreeableness_category'] = users_personality.apply(lambda row: get_personality_category(row.agreeableness, MEAN_2_MALE, STD_2_MALE) if row.gender == "MALE" else get_personality_category(row.agreeableness, MEAN_2_FEMALE, STD_2_FEMALE), axis=1)

users_personality['ipip_conscientiousness_category'] = users_personality.apply(lambda row: get_personality_category(row.conscientiousness, MEAN_3_MALE, STD_3_MALE) if row.gender == "MALE" else get_personality_category(row.conscientiousness, MEAN_3_FEMALE, STD_3_FEMALE), axis=1)

users_personality['ipip_stability_category'] = users_personality.apply(lambda row: get_personality_category(row.stability, MEAN_4_MALE, STD_4_MALE) if row.gender == "MALE" else get_personality_category(row.stability, MEAN_4_FEMALE, STD_4_FEMALE), axis=1)

users_personality['ipip_intellect_category'] = users_personality.apply(lambda row: get_personality_category(row.intellect, MEAN_5_MALE, STD_5_MALE) if row.gender == "MALE" else get_personality_category(row.intellect, MEAN_5_FEMALE, STD_5_FEMALE), axis=1)

In [142]:
users_personality = users_personality[users_personality.columns.drop(list(users_personality.filter(regex='ipip\[SQ')))]
users_personality.to_csv("data/surveys/personality.csv")
users_personality.to_pickle("data/surveys/personality.pkl")
users_personality

,user_id,type,submitdate,extraversion,agreeableness,conscientiousness,stability,intellect,gender,ipip_extraversion_category,ipip_agreeableness_category,ipip_conscientiousness_category,ipip_stability_category,ipip_intellect_category
0,621e2e8e67b776a24055b564,bfpt,2021-05-31,21.0,33.0,45.0,42.0,40.0,MALE,LOW,LOW,HIGH,HIGH,HIGH
1,621e2eaf67b776a2406b14ac,bfpt,2021-11-29,32.0,45.0,30.0,18.0,41.0,FEMALE,AVERAGE,HIGH,AVERAGE,LOW,HIGH
2,621e2ed667b776a24085d8d1,bfpt,2021-06-07,40.0,43.0,22.0,28.0,34.0,FEMALE,HIGH,AVERAGE,LOW,AVERAGE,AVERAGE
3,621e2f3967b776a240c654db,bfpt,2021-05-31,25.0,34.0,30.0,39.0,37.0,MALE,LOW,AVERAGE,LOW,HIGH,AVERAGE
4,621e2f6167b776a240e082a9,bfpt,2021-06-01,41.0,41.0,30.0,34.0,30.0,FEMALE,HIGH,AVERAGE,AVERAGE,HIGH,LOW
5,621e2f7a67b776a240f14425,bfpt,2021-05-31,29.0,38.0,43.0,39.0,35.0,MALE,AVERAGE,AVERAGE,HIGH,HIGH,AVERAGE
6,621e2f9167b776a240011ccb,bfpt,2021-11-28,21.0,45.0,31.0,18.0,39.0,FEMALE,LOW,HIGH,AVERAGE,LOW,HIGH
7,621e2fb367b776a24015accd,bfpt,2021-05-31,37.0,36.0,29.0,27.0,41.0,MALE,HIGH,AVERAGE,LOW,LOW,HIGH
8,621e2fce67b776a240279baa,bfpt,2021-05-31,34.0,42.0,35.0,27.0,35.0,MALE,HIGH,HIGH,AVERAGE,LOW,AVERAGE
9,621e2ff067b776a2403eb737,bfpt,2021-11-29,33.0,44.0,23.0,11.0,30.0,FEMALE,AVERAGE,HIGH,LOW,LOW,LOW


Integrating BREQ-2 Type Responses

In [143]:
users_breq = pd.DataFrame(columns=["user_id", "data"])
for user in users:
    user_data = pd.DataFrame(list(
        db.surveys.find({"$and": [
            {"type": "breq"},
            {"user_id": user}
        ]},
            {"_id": 0}
        )
    ))
    users_breq = pd.concat([users_breq, user_data], axis=0)
users_breq["submitdate"] = users_breq["data"].apply(lambda d: d["submitdate"])
users_breq["submitdate"] = pd.to_datetime(users_breq["submitdate"], infer_datetime_format=True).dt.date

for engage in range(1,20):
    if engage < 10:
        col_name = "engage[SQ00{}]".format(engage)
    else:
        col_name = "engage[SQ0{}]".format(engage)
    users_breq[col_name] = users_breq["data"].apply(lambda d: d[col_name])
users_breq.drop(['data'], axis=1, inplace=True)

# find a mean per factor per user
users_breq["breq_amotivation"] = users_breq[["engage[SQ005]", "engage[SQ009]", "engage[SQ012]", "engage[SQ019]"]].mean(axis=1)
users_breq["breq_external_regulation"] = users_breq[["engage[SQ001]", "engage[SQ006]", "engage[SQ011]", "engage[SQ016]"]].mean(axis=1)
users_breq["breq_introjected_regulation"] = users_breq[["engage[SQ002]", "engage[SQ007]", "engage[SQ013]"]].mean(axis=1)
users_breq["breq_identified_regulation"] = users_breq[["engage[SQ003]", "engage[SQ008]", "engage[SQ014]", "engage[SQ017]"]].mean(axis=1)
users_breq["breq_intrinsic_regulation"] = users_breq[["engage[SQ004]", "engage[SQ010]", "engage[SQ015]", "engage[SQ018]"]].mean(axis=1)
temp = users_breq[["breq_amotivation", "breq_external_regulation", "breq_introjected_regulation", "breq_identified_regulation", "breq_intrinsic_regulation"]].agg(['idxmax','max'], axis=1).mask(lambda x: x['max'].eq(0))
users_breq.loc[:, "breq_self_determination"] = temp.loc[:, "idxmax"]
users_breq["breq_self_determination"].replace("breq_", "", inplace=True, regex=True)

# drop duplicates
users_breq.drop_duplicates(subset=["user_id", "submitdate"], keep="last", inplace=True)
users_breq = users_breq[users_breq.columns.drop(list(users_breq.filter(regex='engage')))]
users_breq.to_csv("data/surveys/breq.csv")
users_breq.to_pickle("data/surveys/breq.pkl")
users_breq

,user_id,type,submitdate,breq_amotivation,breq_external_regulation,breq_introjected_regulation,breq_identified_regulation,breq_intrinsic_regulation,breq_self_determination
0,621e2e8e67b776a24055b564,breq,2021-05-31,1.00,1.00,1.000000,3.50,4.25,intrinsic_regulation
1,621e2e8e67b776a24055b564,breq,2021-07-26,1.00,1.00,1.666667,3.50,4.00,intrinsic_regulation
0,621e2eaf67b776a2406b14ac,breq,2021-11-29,1.25,1.00,1.666667,4.50,3.75,identified_regulation
1,621e2eaf67b776a2406b14ac,breq,2022-01-17,1.00,1.25,2.333333,4.00,3.50,identified_regulation
0,621e2ed667b776a24085d8d1,breq,2021-06-07,2.75,2.00,3.000000,2.50,1.00,introjected_regulation
...,...,...,...,...,...,...,...,...,...
0,621e36c267b776a240ba2756,breq,2021-05-31,1.00,1.00,1.000000,3.75,4.75,intrinsic_regulation
0,621e36f967b776a240e5e7c9,breq,2021-05-31,1.00,1.75,4.666667,4.00,3.25,introjected_regulation
1,621e36f967b776a240e5e7c9,breq,2021-07-26,1.00,1.75,5.000000,4.75,3.25,introjected_regulation
0,621e375b67b776a240290cdc,breq,2021-06-01,1.00,1.00,1.666667,3.50,2.75,identified_regulation


Integrating TTM Responses

In [144]:
users_ttm = pd.DataFrame(columns=["user_id", "data"])
for user in users:
    user_data = pd.DataFrame(list(
        db.surveys.find({"$and": [
            {"type": "ttmspbf"},
            {"user_id": user}
        ]},
            {"_id": 0}
        )
    ))

    users_ttm = pd.concat([users_ttm, user_data], axis=0)

users_ttm["submitdate"] = users_ttm["data"].apply(lambda d: d["submitdate"])
users_ttm["stage"] = users_ttm["data"].apply(lambda d: d["stage"])
users_ttm["submitdate"] = pd.to_datetime(users_ttm["submitdate"], infer_datetime_format=True).dt.date

for engage in range(2,32):
    if engage < 10:
        col_name = "processes[SQ00{}]".format(engage)
        new_col_name = "processes[SQ00{}]".format(engage-1)
    else:
        col_name = "processes[SQ0{}]".format(engage)
        if engage == 10:
            new_col_name = "processes[SQ00{}]".format(engage-1)
        else:
            new_col_name = "processes[SQ0{}]".format(engage-1)
    users_ttm[new_col_name] = users_ttm["data"].apply(lambda d: d[col_name])

users_ttm.drop(['data'], axis=1, inplace=True)

def define_stage_of_change(response):
    if response == "No, and I do not intend to do regular physical activity in the next 6 months.":
        return "Precontemplation"
    if response == "No, but I intend to do regular physical activity in the next 6 months.":
        return "Contemplation"
    if response == "No, but I intend to do regular physical activity in the next 30 days.":
        return "Preparation"
    if response == "Yes, I have been doing physical activity regularly, but for less than 6 months.":
        return "Action"
    return "Maintenance"

users_ttm["stage"] = users_ttm.stage.apply(lambda response: define_stage_of_change(response))

# drop duplicates
users_ttm.drop_duplicates(subset=["user_id", "submitdate"], keep="last", inplace=True)

users_ttm["ttm_consciousness_raising"] = users_ttm[["processes[SQ001]","processes[SQ011]", "processes[SQ021]"]].mean(axis=1)
users_ttm["ttm_dramatic_relief"] = users_ttm[["processes[SQ002]","processes[SQ012]", "processes[SQ022]"]].mean(axis=1)
users_ttm["ttm_environmental_reevaluation"] = users_ttm[["processes[SQ003]","processes[SQ013]", "processes[SQ023]"]].mean(axis=1)
users_ttm["ttm_self_reevaluation"] = users_ttm[["processes[SQ004]","processes[SQ014]", "processes[SQ024]"]].mean(axis=1)
users_ttm["ttm_social_liberation"] = users_ttm[["processes[SQ005]","processes[SQ015]", "processes[SQ025]"]].mean(axis=1)
users_ttm["ttm_counterconditioning"] = users_ttm[["processes[SQ006]","processes[SQ016]", "processes[SQ026]"]].mean(axis=1)
users_ttm["ttm_helping_relationships"] = users_ttm[["processes[SQ007]","processes[SQ017]", "processes[SQ027]"]].mean(axis=1)
users_ttm["ttm_reinforcement_management"] = users_ttm[["processes[SQ008]","processes[SQ018]", "processes[SQ028]"]].mean(axis=1)
users_ttm["ttm_self_liberation"] = users_ttm[["processes[SQ009]","processes[SQ019]", "processes[SQ029]"]].mean(axis=1)
users_ttm["ttm_stimulus_control"] = users_ttm[["processes[SQ010]","processes[SQ020]", "processes[SQ030]"]].mean(axis=1)

users_ttm = users_ttm[users_ttm.columns.drop(list(users_ttm.filter(regex='processes')))]
users_ttm.to_csv("data/surveys/ttm.csv")
users_ttm.to_pickle("data/surveys/ttm.pkl")
users_ttm

,user_id,type,submitdate,stage,ttm_consciousness_raising,ttm_dramatic_relief,ttm_environmental_reevaluation,ttm_self_reevaluation,ttm_social_liberation,ttm_counterconditioning,ttm_helping_relationships,ttm_reinforcement_management,ttm_self_liberation,ttm_stimulus_control
0,621e2e8e67b776a24055b564,ttmspbf,2021-07-26,Maintenance,1.333333,2.666667,2.333333,4.333333,4.000000,4.000000,4.000000,4.000000,4.000000,1.666667
0,621e2eaf67b776a2406b14ac,ttmspbf,2021-11-29,Contemplation,3.333333,2.666667,4.000000,4.666667,4.000000,2.333333,3.000000,4.333333,3.333333,1.000000
1,621e2eaf67b776a2406b14ac,ttmspbf,2022-01-17,Action,3.666667,2.333333,4.333333,4.333333,3.333333,1.666667,3.000000,4.333333,3.333333,3.000000
0,621e2ed667b776a24085d8d1,ttmspbf,2021-06-07,Preparation,1.000000,2.000000,3.000000,3.333333,3.333333,1.000000,1.000000,3.333333,1.666667,1.000000
1,621e2ed667b776a24085d8d1,ttmspbf,2021-07-30,Contemplation,1.666667,2.333333,4.000000,3.333333,3.666667,1.333333,1.000000,2.333333,2.333333,1.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,621e36c267b776a240ba2756,ttmspbf,2021-05-31,Action,2.666667,1.000000,3.000000,4.333333,3.666667,4.000000,3.333333,4.000000,3.666667,3.000000
0,621e36f967b776a240e5e7c9,ttmspbf,2021-05-31,Action,2.000000,4.333333,3.333333,5.000000,3.000000,3.333333,1.666667,3.666667,4.000000,3.333333
1,621e36f967b776a240e5e7c9,ttmspbf,2021-07-26,Action,4.000000,3.666667,3.666667,5.000000,3.000000,3.000000,1.666667,4.666667,4.000000,3.000000
0,621e375b67b776a240290cdc,ttmspbf,2021-06-01,Action,2.333333,1.666667,1.000000,2.666667,5.000000,1.000000,3.000000,4.666667,2.666667,1.333333


Integrating STAI Data

In [145]:
users_stai = pd.DataFrame(columns=["user_id", "data"])
for user in users:
    user_data = pd.DataFrame(list(
        db.surveys.find({"$and": [
            {"type": "stai"},
            {"user_id": user}
        ]},
            {"_id": 0}
        )
    ))

    users_stai = pd.concat([users_stai, user_data], axis=0)

users_stai["submitdate"] = users_stai["data"].apply(lambda d: d["submitdate"])
users_stai["submitdate"] = pd.to_datetime(users_stai["submitdate"], infer_datetime_format=True).dt.date

for engage in range(1,21):
    if engage < 10:
        col_name = "STAI[SQ00{}]".format(engage)
    else:
        col_name = "STAI[SQ0{}]".format(engage)
    users_stai[col_name] = users_stai["data"].apply(lambda d: d[col_name])

users_stai.drop(['data'], axis=1, inplace=True)

# convert 5-likert to 4-likert scale (mistakenly the STAI scale was distributed with a 5-likert, but it's originally 4-likert)
def convert_5_to_4_likert(x):
    return (4 - 1) * (x - 1) / (5 - 1) + 1

users_stai.iloc[:, 3:] = users_stai.iloc[:, 3:].apply(lambda x: convert_5_to_4_likert(x))

def proper_round(num, dec=0):
    num = str(num)[:str(num).index('.')+dec+2]
    if num[-1]>='5':
      a = num[:-2-(not dec)]       # integer part
      b = int(num[-2-(not dec)])+1 # decimal part
      return float(a)+b**(-dec+1) if a and b == 10 else float(a+str(b))
    return float(num[:-1])

for col in users_stai.iloc[:, 3:].columns:
    users_stai[col] = users_stai[col].apply(lambda x: proper_round(x))

# Based on the scoring document: https://oml.eular.org/sysModules/obxOML/docs/id_150/State-Trait-Anxiety-Inventory.pdf, some questions are reversed in STAI
stai_reversed = ["STAI[SQ001]", "STAI[SQ002]", "STAI[SQ005]", "STAI[SQ008]", "STAI[SQ010]", "STAI[SQ011]", "STAI[SQ015]", "STAI[SQ016]", "STAI[SQ019]", "STAI[SQ020]"]
for col in users_personality.columns:
    # inversing scores for the minus keyed items; the plus keyed items stay as is
    if col in stai_reversed:
        users_stai[col] = users_stai[col].apply(lambda score: inverse_score(score, 1, 4))

# to calculate the total stress score simply sum per row
users_stai['stai_stress'] = users_stai.iloc[:, 3:].sum(axis=1)

mean_stai = users_stai['stai_stress'].mean()
std_stai = users_stai['stai_stress'].std()

def get_stai_category(score):
    if score < mean_stai-0.5*std_stai:
        return "Below average"
    if score > mean_stai+0.5*std_stai:
        return "Above average"
    return "Average"

users_stai['stai_stress_category'] = users_stai['stai_stress'].apply(lambda score: get_stai_category(score))

# drop duplicates
users_stai.drop_duplicates(subset=["user_id", "submitdate"], inplace=True, keep="last")
users_stai = users_stai[users_stai.columns.drop(list(users_stai.filter(regex='STAI\[SQ')))]
users_stai.to_csv("data/surveys/stai.csv")
users_stai.to_pickle("data/surveys/stai.pkl")
users_stai

,user_id,type,submitdate,stai_stress,stai_stress_category
0,621e2e8e67b776a24055b564,stai,2021-05-31,45.0,Below average
1,621e2e8e67b776a24055b564,stai,2021-06-07,46.0,Average
3,621e2e8e67b776a24055b564,stai,2021-07-19,46.0,Average
4,621e2e8e67b776a24055b564,stai,2021-07-26,46.0,Average
5,621e2e8e67b776a24055b564,stai,2021-06-28,54.0,Above average
...,...,...,...,...,...
0,621e375b67b776a240290cdc,stai,2021-06-08,60.0,Above average
1,621e375b67b776a240290cdc,stai,2021-07-06,60.0,Above average
2,621e375b67b776a240290cdc,stai,2021-06-01,59.0,Above average
3,621e375b67b776a240290cdc,stai,2021-06-22,60.0,Above average


Integrating PANAS scale

In [146]:
users_panas = pd.DataFrame(columns=["user_id", "data"])
for user in users:
    user_data = pd.DataFrame(list(
        db.surveys.find({"$and": [
            {"type": "panas"},
            {"user_id": user}
        ]},
            {"_id": 0}
        )
    ))

    users_panas = pd.concat([users_panas, user_data], axis=0)

users_panas["submitdate"] = users_panas["data"].apply(lambda d: d["submitdate"])
users_panas["submitdate"] = pd.to_datetime(users_panas["submitdate"], infer_datetime_format=True).dt.date

for p in range(1,21):
    if p < 10:
        col_name = "P1[SQ00{}]".format(p)
    else:
        col_name = "P1[SQ0{}]".format(p)
    users_panas[col_name] = users_panas["data"].apply(lambda d: d[col_name])

users_panas.drop(['data'], axis=1, inplace=True)

# drop duplicates
users_panas.drop_duplicates(subset=["user_id", "submitdate"], inplace=True, keep="last")

positive = ["P1[SQ001]", "P1[SQ003]", "P1[SQ005]", "P1[SQ009]", "P1[SQ010]", "P1[SQ012]", "P1[SQ014]", "P1[SQ016]", "P1[SQ017]", "P1[SQ019]"]
negative = ["P1[SQ002]", "P1[SQ004]", "P1[SQ006]", "P1[SQ007]", "P1[SQ008]", "P1[SQ011]", "P1[SQ013]", "P1[SQ015]", "P1[SQ018]", "P1[SQ020]"]
users_panas["positive_affect_score"] = users_panas[positive].sum(axis=1)
users_panas["negative_affect_score"] = users_panas[negative].sum(axis=1)

users_panas = users_panas[users_panas.columns.drop(list(users_panas.filter(regex='P1\[SQ')))]
users_panas.to_csv("data/surveys/panas.csv")
users_panas.to_pickle("data/surveys/panas.pkl")
users_panas

,user_id,type,submitdate,positive_affect_score,negative_affect_score
1,621e2e8e67b776a24055b564,panas,2021-07-26,37,14
2,621e2e8e67b776a24055b564,panas,2021-05-31,38,12
3,621e2e8e67b776a24055b564,panas,2021-06-07,37,12
4,621e2e8e67b776a24055b564,panas,2021-06-28,31,12
5,621e2e8e67b776a24055b564,panas,2021-07-19,37,12
...,...,...,...,...,...
1,621e375367b776a24021e950,panas,2022-01-02,30,33
0,621e375b67b776a240290cdc,panas,2021-06-01,27,29
1,621e375b67b776a240290cdc,panas,2021-06-08,30,30
2,621e375b67b776a240290cdc,panas,2021-06-22,30,30
